# Capstone Project: Oakland Roots SC

Team information(Alphabetize)
- Member: Denggao Jiang
- Project Summary: 

Member information(Alphabetize)
- Family Name: Denggao
- Given Name: Jiang
- ID: ####
- Student email: ##########

Programming Language: Python 3.9 in Jupyter Notebook

Python Libraries used:
- pandas
- numpy
- requests
- json
- csv
- re
- time
- lxml
- os
- calendar
- missingno
- pandas_profiling
- sklearn
- matplotlib
- plotly
- ipywidgets
- warnings
- dash
- jupyter_dash

## Table of Contents <a class="anchor" id="sec_0"></a>

* [Executive Summary](#sec_1)
* [Data introduction](#sec_2)
* [Webscrape and Dataset Retrieve](#sec_3)
 * [Wyscout Dataset](#sec_3.1)
 * [ASA Dataset](#sec_3.2)
 * [TransferMarkt Dataset Webscrape](#sec_3.3)
   * [Get a total of 6 pages of players' information](#sec_3.3.1)
   * [Foreigner player information](#sec_3.3.2)
 * [Sofa Score Dataset Webscrape (2020&2021)](#sec_3.4)
* [Data Manipulation & EDA Analysis](#sec_4)
 * [Position Integration](#sec_4.1)
 * [Wyscout Data Cleaning (Position and Market Value)](#sec_4.2)
 * [ASA Dataset Merge and Data Cleaning](#sec_4.3)
 * [TransferMarkt Data Cleaning](#sec_4.4)
 * [Combine Wyscout and Ratings Dataset](#sec_4.5)
 * [Data Analysis](#sec_4.6)
 * [Data Visualization](#sec_4.7)
* [Features Selection (Wyscout Dataset)](#sec_5)
 * [Split Dataset with Positions](#sec_5.1)
 * [Deploy Random Forest Algorithm](#sec_5.2)
* [Cosine Distance](#sec_6)
 * [Radar Plot](#sec_6.1)
   * [Player comparison engine](#sec_6.1.1)
 * [Another Way Cosine Distance](#sec_6.2)
   * [Position FB -- Most similar players to R. Cannon](#sec_6.2.1)
   * [Position CB -- Most similar players to J. Sands](#sec_6.2.2)
   * [Position CM -- Most similar players to J. Sands & C. Bassett](#sec_6.2.3)
   * [Position AM -- Most similar players to C. Bassett](#sec_6.2.4)
   * [Position CF -- Most similar players to R. Damus & C. Bassett](#sec_6.2.5)
   * [Summary](#sec_6.2.6)
   * [Find Player Information](#sec_6.2.7)
* [Rating Model](#sec_7)
* [Market Value Model](#sec_8)
 * [Market Value Forecast](#sec_8.1)
 * [Market Value supplement](#sec_8.2)
   * [Before supplement](#sec_8.2.1)
   * [After supplement](#sec_8.2.2)
* [Abandon cases](#sec_9)
 * [The Neural Network Model to Predict the Market Value](#sec_9.1)

## 1. Executive Summary <a class="anchor" id="sec_1"></a>
* [Back to menu](#sec_0)

### Problem  
Oakland Roots needs help recruiting talented soccer players to their team. In particular, they are looking to find players with skillsets lacking in the Roots team and at an affordable price.  
### Solution  
With data sourced and scraped from soccer statistics websites, we have provided a comprehensive analysis/plan of attack for Oakland Roots. We delve into player statistics, team dynamics, field layouts, and feature/skillset importance to make better-informed recruitment decisions.  
### Highlights  
Using various analytic techniques, we uncovered resources & statistics that provide the Oakland Roots team strategies to recruit talent. These include 3 main categories (1) Talent-based recruitments (based on specific features of desired players), (2) Similarity-based recruitment (comparing to other players), and (3) Team-based recommendations & blindspots (based on what team dynamics lead to winning championships). More specifically, we this breaks down into these loosely-grouped analytic categories:
  
1. Undervalued Players (Market Value Model)   
2. Similar Players (Cosine Similarity)   
3. Important Features of Valuable Players (Random Forest) 
4. Important Features Per Position 
5. Team-by-Team Comparisons 
6. Statistics by Features 
7. Player Country of Origin (Choropleth)
8. Radar Plots Comparing Most Similar Players

## 2. Data introduction <a class="anchor" id="sec_2"></a>
* [Back to menu](#sec_0)

### Wyscout dataset (Wy)

The Wyscout dataset is retrieved from the Wyscout data platform as provided by the client (Jordan). Wyscout is the largest soccer data database on the internet. It compares players in the USL League One — ranking players by shots, crosses, & successful tackles in the USL Championship Games. Moreover, they have team statistics — correlation stats of formation play & winning rate of each team for multiple or singular games. Statistics range from broad (team/games level) to narrow (players/minute-by-minute level). We acquired 2020 & 2021 datasets on all the players in USL League One — extracting player positions, features, & statistics.

### American Soccer Analysis (ASA)

The American Soccer Analysis (ASA) dataset is downloaded to visualize the data. The dataset contains three separate datasets of xGoals, xPass, and Goals Added (g+). The metric xGoals (xG) assesses the probability (%) of any shot scoring (a goal). Specifically, it quantifies the difficulty of a scoring shot using various predictive metrics — players' goals, key passes, & assists. Likewise, the metric xPass (xP) assesses the probability (%) of a pass being successful (making it to a teammate). Other columns include — passing percentage (%), number of passes (#), passing distance (m), & player's touch percentage of the ball (%). The Goals Added (g+) dataset measures a player's total contribution in attack & defense during gameplay. Each column in the dataset are factors — dribbling, fouling, passing, & receiving — can affect a team's chance to score or overturn possession. 

### Transfermarkt

Transfermarkt is a website where anyone can look up a soccer team with player statistics, game stats, & a player's market value ($). The market value on Transfermarkt is determined by the administrator in charge of each regional league & a data scout. Aforesaid data scout considers factors like (1) the level of the league the player is in, (2) the player's position, (3) age, & (4) recent trading bids from other teams (higher bids increase market value). The market value is helpful for sports managers of a team to get an idea of a player's skill level. We webscraped every player's statistics—including market value—from the website. This dataset was primarily combined with Wyscout's and ASA datasets (matching by player names) for modeling and visualization purposes. This data was then used in a Moneyball-like attempt to identify undervalued players according to their skill level & market value.

### Sofa Score Dataset (2020 & 2021)

We found ratings of each player in the USL League One on a website called SofaScore. This website built an algorithm that quantifies a player's performance by aggregating player statistics into one “rating”. The SofaScore ratings for 2020 and 2021 were scraped from the site. This dataset was merged with Wyscout to construct feature selection, ratings, and a salary model. 

## 3. Webscrape and Dataset Retrieve <a class="anchor" id="sec_3"></a>
* [Back to menu](#sec_0)

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import re
import time
from lxml import etree
import os
import calendar
import missingno as msno
import pandas_profiling as pf

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics.pairwise import cosine_similarity, paired_distances

import matplotlib.pyplot as plt
from plotly import tools
from plotly.subplots import make_subplots
import plotly as py
import plotly.express as px
import plotly.graph_objs as go

import ipywidgets as widgets
py.offline.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings ('ignore')

from dash import Dash, dcc, html, dcc, Input, Output, State
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash

/Users/dg/opt/anaconda3/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



### 3.1 Wyscout Dataset <a class="anchor" id="sec_3.1"></a>
* [Back to menu](#sec_0)

In [2]:
Data_Wy_2020 = pd.read_excel('datasets/Data_Wy_2020.xlsx')
Data_Wy_2021 = pd.read_excel('datasets/Data_Wy_2021.xlsx')

### 3.2 ASA Dataset <a class="anchor" id="sec_3.2"></a>
* [Back to menu](#sec_0)

In [3]:
data_g = pd.read_csv('datasets/american_soccer_analysis_usl1_xgoals_players_2022-03-21.csv').dropna(axis=1, how="all")
data_p = pd.read_csv('datasets/american_soccer_analysis_usl1_xpass_players_2022-03-21.csv').dropna(axis=1, how="all")
data_g_a = pd.read_csv('datasets/american_soccer_analysis_usl1_goals-added_players_2022-03-21.csv').dropna(axis=1, how="all")

### 3.3 TransferMarkt Dataset Webscrape <a class="anchor" id="sec_3.3"></a>
* [Back to menu](#sec_0)

#### 3.3.1 Get a total of 6 pages of players' information <a class="anchor" id="sec_3.3.1"></a>
* [Back to menu](#sec_0)

In [4]:
#Create path
def get_request():
    if not os.path.exists('./datasets'):
        os.mkdir('./datasets')
        
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
    for i in range(1, 7):
        url = 'https://www.transfermarkt.us/usl-league-one/torschuetzenliste/wettbewerb/USC3/ajax/yw1/saison_id/2020/altersklasse/alle/detailpos//plus/1/page/{}?ajax=yw1'.format(i)
        html = requests.get(url, headers=headers).text
        get_parse(html)

In [5]:
def change_time(input_time):
    year = input_time.split(',')[-1].strip()
    month = str(list(calendar.month_abbr).index(input_time.split(' ')[0]))

    if int(month) < 10:
        month = '0' + str(month)

    day = int(input_time.split(',')[0].split(' ')[-1])
    if day < 10:
        day = '0' + str(day)
    get_day = str(year) + '-' + str(month) + '-' + str(day)

    return get_day

In [6]:
def get_parse(html):
    try:
        df = pd.read_csv('./datasets/data_TansferMarkt_2021.csv')['player_href'].values
    except:
        df=''
    
    data = etree.HTML(html)
    year = '2021'
    competition = 'USL1'
    trs = data.xpath('//table[@class="items"]/tbody/tr')
    for tr in trs:
        headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",}
        # Player Details Page
        player_href = 'https://www.transfermarkt.us' + tr.xpath('./td[2]//a/@href')[0].strip()
        if player_href in df:
            pass
        else:
            #Player_name
            player_name = tr.xpath('./td[2]//a/@title')[0].strip()
            
            #Position
            position = tr.xpath('./td[2]/table/tr[2]/td/text()')[0].strip()
            
            #National
            nat = '-'.join(tr.xpath('./td[3]/img/@title'))
            
            #Age
            age = tr.xpath('./td[4]/text()')[0].strip()
            
            #Club
            try:
                club = tr.xpath('./td[5]/a/@title')[0].strip()
            except:
                club = tr.xpath('./td[5]/text()')[0].strip()

            #Appearances
            appearances = tr.xpath('./td[6]/a/text()')[0].strip()
            
            #Assists
            assists = tr.xpath('./td[7]/text()')[0].strip()
            
            #Penalty_kicks
            penalty_kicks = tr.xpath('./td[8]/text()')[0].strip()
            
            #Minutes_played
            minutes_played = tr.xpath('./td[9]/text()')[0].strip().replace('\'', '').replace(',', '').replace('.', '')

            #Minutes_per_goals
            minutes_per_goals = tr.xpath('./td[10]/text()')[0].strip().replace('\'', '').replace(',', '').replace('.', '')

            #goals_per_match
            goals_per_match = tr.xpath('./td[11]/text()')[0].strip()
            
            #Goals
            goals = tr.xpath('./td[12]/a/text()')[0].strip()
            
            # More detail page
            re_html = requests.get(player_href, headers=headers).text
            re_data = etree.HTML(re_html)

            #Birthdate
            try:
                date_br = re.findall("""birthDate".*?class="data-header__content">(.*?)</span>""", re_html, re.S)
                Birthdate = date_br[0].split('(')[0].strip()
                Birthdate = change_time(Birthdate)
            except:
                Birthdate = ''
            
            #Height
            try:
                Height = re.findall("""Height:.*?<span itemprop="height" class="data-header__content">(.*?)</span>""",
                                    re_html, re.S)
                Height = Height[0].replace(',', '.').replace('m', '').strip()
            except:
                Height = ''
            if Height == position:
                Height = ''
            
            #League level
            try:
                League_level = re.findall(
                    """League level:.*?<span class="data-header__content">.*?<img src=".*?" title="United States" alt="United States" class="flaggenrahmen" />(.*?)</span>.*?</span>""",
                    re_html, re.S)[0].strip()
            except:
                League_level = ''
            
            #Salary
            Salary = ''.join(re_data.xpath('//div[@class="data-header__box--small"]/a//text()')[:3]).strip().replace(
                '$','').replace('Th.', '000')
            
            #Salary Date
            try:
                Salary_date = re_data.xpath('//div[@class="data-header__box--small"]/a/p/text()')[0].split(':')[-1].strip()
                Salary_date = change_time(salary_date)
            except:
                Salary_date = ''
            
            #Contract date(until)
            try:
                contract_until = re.findall('regular">Contract expires:</span>.*?bold">(.*?)</span>', re_html, re.S)[0].strip()
                contract_until = change_time(contract_until)
            except:
                contract_until = ''
            
            #Position detail
            try:
                position_detail = re.findall('regular">Position:</span>.*?bold">(.*?)</span>', re_html, re.S)[0].strip()
            except:
                position_detail = ''
                
            #Player agent
            try:
                player_agent = re.findall('regular">Player agent:</span>.*?<a.*?>(.*?)</a>', re_html, re.S)[0].strip()
                if 'img' in player_agent:
                    player_agent = re.findall('regular">Player agent:</span>.*?>(.*?)</.*?>', re_html, re.S)[0].strip().replace('<span>','')
            except:
                player_agent = ''
                
            #Date of last contract extension
            try:
                date_of_last_contract_extension = re.findall('>Date of last contract extension:</span>.*?">(.*?)</span>', re_html, re.S)[0].strip()
                date_of_last_contract_extension = change_time(date_of_last_contract_extension)
            except:
                date_of_last_contract_extension = ''
                
            #Joined date
            try:
                joined = re.findall('regular">Joined:</span>.*?bold">(.*?)</span>', re_html, re.S)[0].strip()
                joined = change_time(joined)
            except:
                joined = ''
                
            #Highest salary -- Highest market value
            try:
                Highest_market_value = re_data.xpath('//div[@class="tm-player-market-value-development__max"]/div[@class="tm-player-market-value-development__max-value"]/text()'
                                                    )[0].strip().replace('$', '').replace('Th.', '000')
            except:
                Highest_market_value = ''
                
            #Date of highest value
            try:
                Date_of_Highest_Value = \
                re_data.xpath('//div[@class="tm-player-market-value-development__max"]/div[2]/text()')[
                    0].strip()
                Date_of_Highest_Value = change_time(Date_of_Highest_Value)
            except:
                Date_of_Highest_Value = ''

            headers = ['Year', 'Competition', 'Player Name', 'Nat.', 'Age', 'Height(Unit:m)', 
                       'Birthdate', 'Club', 'Position', 'Position Detail', 'Appearances', 'Assists', 'Penalty Kicks', 
                       'Minutes Played', 'Minutes Per Goals', 'Goals Per Match', 'Goals', 'League Level',
                       'Salary', 'Salary Date', 'Joined Date', 'Contract Until', 'Player Agent', 'Date of Last Contract Extension', 
                       'Highest Market Value', 'Date of Highest Value', 'player_href']
            
            get_values = [year, competition, player_name, nat, age, Height, 
                          Birthdate, club, position, position_detail, appearances, assists, penalty_kicks, 
                          minutes_played, minutes_per_goals, goals_per_match, goals, League_level, 
                          Salary, Salary_date, joined, contract_until, player_agent, date_of_last_contract_extension,
                          Highest_market_value, Date_of_Highest_Value, player_href]
            

            with open('./datasets/data_TansferMarkt_2021.csv', 'a', encoding='utf_8_sig', newline='') as csvfile:
                writer = csv.writer(csvfile)
                with open('./datasets/data_TansferMarkt_2021.csv', 'r', encoding='utf_8_sig', newline='') as f:
                    reader = csv.reader((line.replace('\0', '') for line in f))
                    if not [row for row in reader]:
                        writer.writerow(headers)
                        writer.writerow(get_values)
                    else:
                        writer.writerow(get_values)
            time.sleep(0.1)

#### 3.3.2 Foreigner player information <a class="anchor" id="sec_3.3.2"></a>
* [Back to menu](#sec_0)

In [7]:
def foreigners_request():
    if not os.path.exists('./datasets'):
        os.mkdir('./datasets')

    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",}
    url = 'https://www.transfermarkt.us/usl-league-one/gastarbeiter/wettbewerb/USC3/saison_id/2020'
    html = requests.get(url, headers=headers).text
    foreigners_parse(html)

In [8]:
def foreigners_parse(html):
    data = etree.HTML(html)
    year = '2021'
    competition = 'USL1'
    trs = data.xpath('//div[@class="responsive-table"]/div[@class="grid-view"]/table/tbody/tr')
    print("Foreigners Part")
    for tr in trs:
        headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}

        player_href = 'https://www.transfermarkt.us' + tr.xpath('./td[4]//td[@class="hauptlink"]/a/@href')[0].strip()
        
        #Palyer_name
        player_name = tr.xpath('./td[4]//a/@title')[0].strip()
        
        #Position
        position = tr.xpath('./td[4]/table/tr[2]/td/text()')[0].strip()
        
        #National
        nat = tr.xpath('./td[1]/a/@title')[0]
        
        #Club
        try:
            club = tr.xpath('./td[5]/a/@title')[0].strip()
        except:
            club = tr.xpath('./td[5]/text()')[0].strip()
        
        #Age. The age attribute is handled in the player's personal details page, in the second subsection.
        
        #Personal pages
        #Appearences; Assists; Penalty_kicks; Minutes_played; Minutes_per_goals; Goals_per_match; Goals
        
        re_html = requests.get(player_href, headers=headers).text
        re_data = etree.HTML(re_html)
        appearances, assists, penalty_kicks, minutes_played, minutes_per_goals, goals_per_match, goals = '', '', '', '', '', '', ''

        trs = re_data.xpath('//div[@class="grid-view"]/table/tbody/tr')
        for tr in trs:
            try:
                if tr.xpath('./td[2]/a/@title')[0] == 'USL1':
                    appearances = tr.xpath('./td[3]/a/text()')[0].strip()
                    assists = tr.xpath('./td[5]/text()')[0].strip()
                    penalty_kicks = ''
                    minutes_played = tr.xpath('./td[7]/text()')[0].strip().replace('\'', '').replace(',', '').replace('.', '')
                    minutes_per_goals = tr.xpath('./td[6]/text()')[0].strip()
                    goals_per_match = ''
                    goals = tr.xpath('./td[4]/text()')[0].strip()
                    break
            except:
                pass

        #Birthdate
        try:
            date_br = re.findall("""birthDate".*?class="data-header__content">(.*?)</span>""", re_html, re.S)
            Birthdate = date_br[0].split('(')[0].strip()
            Birthdate = change_time(Birthdate)
        except:
            Birthdate = ''
        
        #Height
        try:
            Height = re.findall("""Height:.*?<span itemprop="height" class="data-header__content">(.*?)</span>""",
                                re_html, re.S)
            Height = Height[0].replace(',', '.').replace('m', '').strip()
        except:
            Height = ''
        
        #League level
        try:
            League_level = re.findall(
                """League level:.*?<span class="data-header__content">.*?<img src=".*?" title="United States" alt="United States" class="flaggenrahmen" />(.*?)</span>.*?</span>""",
                re_html, re.S)[0].strip()
        except:
            League_level = ''
  
        #Salary
        Salary = ''.join(re_data.xpath('//div[@class="data-header__box--small"]/a//text()')[:3]).strip().replace(
            '$','').replace('Th.', '000')
        
        #Salary Date
        try:
            Salary_date = re_data.xpath('//div[@class="data-header__box--small"]/a/p/text()')[0].split(':')[-1].strip()
            Salary_date = change_time(salary_date)
        except:
            Salary_date = ''
            
        #Contract date(until)
        try:
            contract_until = re.findall('regular">Contract expires:</span>.*?bold">(.*?)</span>', re_html, re.S)[
                0].strip()
            contract_until = change_time(contract_until)
        except:
            contract_until = ''

        #Position detail
        try:
            position_detail = re.findall('regular">Position:</span>.*?bold">(.*?)</span>', re_html, re.S)[0].strip()
        except:
            position_detail = ''
        
        #Player agent
        try:
            player_agent = re.findall('regular">Player agent:</span>.*?<a.*?>(.*?)</a>', re_html, re.S)[0].strip()
            if 'img' in player_agent:
                player_agent = re.findall('regular">Player agent:</span>.*?>(.*?)</.*?>', re_html, re.S)[0].strip().replace('<span>','')
        except:
            player_agent = ''
            
        #Age
        try:
            age = re.findall('regular">Age:</span>.*?bold">(.*?)</span>', re_html, re.S)[0].strip()
        except:
            age = ''
        
        #Date of last contract extension
        try:
            date_of_last_contract_extension = re.findall('>Date of last contract extension:</span>.*?">(.*?)</span>', re_html, re.S)[0].strip()
            date_of_last_contract_extension = change_time(date_of_last_contract_extension)
        except:
            date_of_last_contract_extension = ''
        
        #Joined date
        try:
            joined = re.findall('regular">Joined:</span>.*?bold">(.*?)</span>', re_html, re.S)[0].strip()
            joined = change_time(joined)
        except:
            joined = ''
            
        #Highest salary -- Highest market value
        try:
            Highest_market_value = re_data.xpath(
                '//div[@class="tm-player-market-value-development__max"]/div[@class="tm-player-market-value-development__max-value"]/text()')[
                0].strip().replace('$', '').replace('Th.', '000')
        except:
            Highest_market_value = ''
        
        #Date of highest value
        try:
            Date_of_Highest_Value = re_data.xpath('//div[@class="tm-player-market-value-development__max"]/div[2]/text()')[
                    0].strip()
            Date_of_Highest_Value = change_time(Date_of_Highest_Value)
        except:
            Date_of_Highest_Value = ''
        
        

        headers = ['Year', 'Competition', 'Player Name', 'Nat.', 'Age', 'Height(Unit:m)', 
                       'Birthdate', 'Club', 'Position', 'Position Detail', 'Appearances', 'Assists', 'Penalty Kicks', 
                       'Minutes Played', 'Minutes Per Goals', 'Goals Per Match', 'Goals', 'League Level',
                       'Salary', 'Salary Date', 'Joined Date', 'Contract Until', 'Player Agent', 'Date of Last Contract Extension', 
                       'Highest Market Value', 'Date of Highest Value', 'player_href']
            
        if Height == position or Height == position_detail:
            Height = ''
        get_values = [year, competition, player_name, nat, age, Height, 
                      Birthdate, club, position, position_detail, appearances, assists, penalty_kicks, 
                      minutes_played, minutes_per_goals, goals_per_match, goals, League_level, 
                      Salary, Salary_date, joined, contract_until, player_agent, date_of_last_contract_extension,
                      Highest_market_value, Date_of_Highest_Value, player_href]

        with open('./datasets/data_TansferMarkt_2021.csv', 'a', encoding='utf_8_sig',newline='') as csvfile:
            writer = csv.writer(csvfile)
            with open('./datasets/data_TansferMarkt_2021.csv', 'r', encoding='utf_8_sig',newline='') as f:
                reader = csv.reader((line.replace('\0', '') for line in f))
                if not [row for row in reader]:
                    writer.writerow(headers)
                    writer.writerow(get_values)
                else:
                    writer.writerow(get_values)
        time.sleep(0.1)

In [ ]:
pd.read_csv('datasets/data_TansferMarkt_2021.csv')

### 3.4 Sofa Score Dataset Webscrape (2020&2021) <a class="anchor" id="sec_3.4"></a>
* [Back to menu](#sec_0)

In [ ]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}

In [ ]:
def Infor(year,pagenum):
    if year == '2021':
        m_number = '36019'
    elif year == '2020':
        m_number = '26862'
    page = int(pagenum)*20
    url = 'https://api.sofascore.com/api/v1/unique-tournament/13362/season/'+str(m_number)+'/statistics?limit=20&order=-rating&offset='+str(page)+'&group=summary'
    res = requests.get(url=url,headers=headers).text
    res = json.loads(res)
    results = res['results']
    goals = []
    successfulDribbles = []
    tackles = []
    assists = []
    accuratePassesPercentages = []
    ratings =[]
    players =[]
    teams =[]
    for r in results:
        goal = r['goals']
        goals.append(goal)
        
        successfulDribble = r['successfulDribbles']
        successfulDribbles.append(successfulDribble)
        
        tackle = r['tackles']
        tackles.append(tackle)
        
        assist = r['assists']
        assists.append(assist)
        
        accuratePassesPercentage = r['accuratePassesPercentage']
        accuratePassesPercentages.append(accuratePassesPercentage)
        
        rating = r['rating']
        ratings.append(rating)
        
        player = r['player']['name']
        players.append(player)
        
        team = r['team']['name']
        teams.append(team)
    
    return goals,successfulDribbles,tackles,assists,accuratePassesPercentages,ratings,players,teams

In [ ]:
# year = '2021'
# goals = []
# successfulDribbles=[]
# tackles = []
# assists= []
# accuratePassesPercentages=[]
# ratings = []
# players = []
# teams = []
# for i in range(17):
#     goal,successfulDribble,tackle,assist,accuratePassesPercentage,rating,player,team =Infor(year,i)

#     for (g,s,ta,ass) in zip(goal,successfulDribble,tackle,assist):
#         goals.append(g)
#         successfulDribbles.append(s)
#         tackles.append(ta)
#         assists.append(ass)
#     for (ac,r,p,te) in zip(accuratePassesPercentage,rating,player,team):
#         accuratePassesPercentages.append(ac)
#         ratings.append(r)
#         players.append(p)
#         teams.append(te)
# dic = {
#     'Team':teams,
#     'Player':players,
#     'Goal':goals,
#     'SuccessfulDribbles':successfulDribbles,
#     'Tackles':tackles,
#     'Assists':assists,
#     'AccuratePassesPercentages':accuratePassesPercentages,
#     'Rating':ratings
# }

# df1 = pd.DataFrame(dic)
# df1.to_csv('datasets/Data_SofaScore_2021.csv')
# pd.read_csv('datasets/Data_SofaScore_2021.csv')

In [ ]:
# year = '2020'
# goals = []
# successfulDribbles=[]
# tackles = []
# assists= []
# accuratePassesPercentages=[]
# ratings = []
# players = []
# teams = []
# for i in range(14):
#     goal,successfulDribble,tackle,assist,accuratePassesPercentage,rating,player,team =Infor(year,i)

#     for (g,s,ta,ass,ac,r,p,te) in zip(goal,successfulDribble,tackle,assist,accuratePassesPercentage,rating,player,team):
#         goals.append(g)
#         successfulDribbles.append(s)
#         tackles.append(ta)
#         assists.append(ass)
#         accuratePassesPercentages.append(ac)
#         ratings.append(r)
#         players.append(p)
#         teams.append(te)
# dic = {
#     'Team':teams,
#     'Player':players,
#     'Goal':goals,
#     'SuccessfulDribbles':successfulDribbles,
#     'Tackles':tackles,
#     'Assists':assists,
#     'AccuratePassesPercentages':accuratePassesPercentages,
#     'Rating':ratings
# }

# df2 = pd.DataFrame(dic)
# df2.to_csv('datasets/Data_SofaScore_2020.csv')
# pd.read_csv('datasets/Data_SofaScore_2020.csv')

## 4. Data Manipulation & EDA Analysis <a class="anchor" id="sec_4"></a>
* [Back to menu](#sec_0)

### 4.1 Position Integration <a class="anchor" id="sec_4.1"></a>
* [Back to menu](#sec_0)

In [ ]:
FB = ['RB', 'RWB', 'LB', 'LWB']                               
CB = ['LCB', 'RCB', 'CH', 'LH', 'RH','CB']                 
CM = ['CDM', 'CM', 'M', 'DMF','RDMF','LDMF','DM','RCMF']  
AM = ['AMC', 'AMF', 'LAMF', 'RAMF', 'AML', 'AMR', 'CAM','AM']
W = ['WF','LWF','RWF','RM','RW','LM','LW','RS','LS']
CF = ['CF', 'LF', 'RF', 'ST','SS','S','']
GK = ['GK','G']

### 4.2 Wyscout Data Cleaning (Position and Market Value) <a class="anchor" id="sec_4.2"></a>
* [Back to menu](#sec_0)

In [ ]:
Data_Wy_2020[['Player', 'Position', 'Market value']].head(5)

In [ ]:
## Market Value to USD, value*1.07
Data_Wy_2020['Market value'] = [i*1.07 for i in Data_Wy_2020['Market value']]
Data_Wy_2021['Market value'] = [i*1.07 for i in Data_Wy_2021['Market value']]

## Classify into 7 positions
Data_Wy_2020['FB'] = ['FB' if len(set(i.replace(' ','').split(',')) & set(FB))>0 else ''  for i in Data_Wy_2020['Position']]
Data_Wy_2020['CB'] = ['CB' if len(set(i.replace(' ','').split(',')) & set(CB))>0 else ''  for i in Data_Wy_2020['Position']]
Data_Wy_2020['CM'] = ['CM' if len(set(i.replace(' ','').split(',')) & set(CM))>0 else ''  for i in Data_Wy_2020['Position']]
Data_Wy_2020['AM'] = ['AM' if len(set(i.replace(' ','').split(',')) & set(AM))>0 else ''  for i in Data_Wy_2020['Position']]
Data_Wy_2020['W'] = ['W' if len(set(i.replace(' ','').split(',')) & set(W))>0 else ''  for i in Data_Wy_2020['Position']]
Data_Wy_2020['CF'] = ['CF' if len(set(i.replace(' ','').split(',')) & set(CF))>0 else ''  for i in Data_Wy_2020['Position']]
Data_Wy_2020['GK'] = ['GK' if len(set(i.replace(' ','').split(',')) & set(GK))>0 else ''  for i in Data_Wy_2020['Position']]
Data_Wy_2020['Position'] = [",".join([s for s in [Data_Wy_2020['FB'][i],Data_Wy_2020['CB'][i],Data_Wy_2020['CM'][i],Data_Wy_2020['AM'][i],
                                    Data_Wy_2020['W'][i],Data_Wy_2020['CF'][i],Data_Wy_2020['GK'][i]] if s !='']) for i in range(len(Data_Wy_2020['Position']))]

Data_Wy_2021['FB'] = ['FB' if len(set(i.replace(' ','').split(',')) & set(FB))>0 else ''  for i in Data_Wy_2021['Position']]
Data_Wy_2021['CB'] = ['CB' if len(set(i.replace(' ','').split(',')) & set(CB))>0 else ''  for i in Data_Wy_2021['Position']]
Data_Wy_2021['CM'] = ['CM' if len(set(i.replace(' ','').split(',')) & set(CM))>0 else ''  for i in Data_Wy_2021['Position']]
Data_Wy_2021['AM'] = ['AM' if len(set(i.replace(' ','').split(',')) & set(AM))>0 else ''  for i in Data_Wy_2021['Position']]
Data_Wy_2021['W'] = ['W' if len(set(i.replace(' ','').split(',')) & set(W))>0 else ''  for i in Data_Wy_2021['Position']]
Data_Wy_2021['CF'] = ['CF' if len(set(i.replace(' ','').split(',')) & set(CF))>0 else ''  for i in Data_Wy_2021['Position']]
Data_Wy_2021['GK'] = ['GK' if len(set(i.replace(' ','').split(',')) & set(GK))>0 else ''  for i in Data_Wy_2021['Position']]
Data_Wy_2021['Position'] = [",".join([s for s in [Data_Wy_2021['FB'][i],Data_Wy_2021['CB'][i],Data_Wy_2021['CM'][i],Data_Wy_2021['AM'][i],
                                    Data_Wy_2021['W'][i],Data_Wy_2021['CF'][i],Data_Wy_2021['GK'][i]] if s !='']) for i in range(len(Data_Wy_2021['Position']))]

Data_Wy_2020.to_csv('datasets/new/Data_Wy_2020_new.csv',index = 0)
Data_Wy_2021.to_csv('datasets/new/Data_Wy_2021_new.csv',index = 0)

In [ ]:
Data_Wy_2020[['Player', 'Position', 'Market value', 'FB', 'CB', 'CM', 'AM', 'W', 'CF', 'GK']].head(5)

### 4.3 ASA Dataset Merge and Data Cleaning <a class="anchor" id="sec_4.3"></a>
* [Back to menu](#sec_0)

In [ ]:
data_g.head(5)

In [ ]:
data_p.head(5)

In [ ]:
data_g_a.head(5)

In [ ]:
# sort dataset with column 'player', and reset index
data_g = data_g.sort_values(by='Player')
data_p = data_p.sort_values(by='Player')
data_g_a = data_g_a.sort_values(by='Player')

data_g.index = range(len(data_g))
data_p.index = range(len(data_p))
data_g_a.index = range(len(data_g_a))

In [ ]:
data_g.head(5)

In [ ]:
data_p.head(5)

In [ ]:
data_g_a.head(5)

In [ ]:
for i in [data_g,data_p,data_g_a]:
    print(i.shape)

In [ ]:
# Determine whether the column ‘player’ of two datasets are the same
df = pd.concat([data_g['Player'], data_p['Player']], axis=1)
df['result'] = np.where(data_g['Player'] == data_p['Player'], 'Same', 'Not Same')
df.head(5)

In [ ]:
# same
df.groupby('result').agg('count')

In [ ]:
# merge 2 dataset
data = pd.merge(data_g, data_p, on=['Player','Team','Season','Position','Minutes'])
print('Columns of merged 2 dataset', data.columns.tolist())

In [ ]:
# players in xgoal and xpass, not in goal_added
e=[]
for i in data['Player'].to_list():
    if i in data_g_a['Player'].to_list():
        continue
    else:
        e.append(i)

print(e)

In [ ]:
# merge 3 dataset
data = pd.merge(data, data_g_a, how='left', on=['Player','Team','Season','Position','Minutes'])
data

In [ ]:
# double check
f=data['Player'][data.isnull().T.any()]
print(f)

In [ ]:
data.to_csv('datasets/data_ASA.csv', index = False)

### 4.4 TransferMarkt Data Cleaning <a class="anchor" id="sec_4.4"></a>
* [Back to menu](#sec_0)

In [ ]:
data_TansferMarkt_2021 = pd.read_csv('datasets/data_TansferMarkt_2021.csv')
data_TansferMarkt_2021[['Player Name', 'Position']].head(5)

In [ ]:
data_TansferMarkt_2021 = pd.read_csv('datasets/data_TansferMarkt_2021.csv')

index = re.compile(r"[A-Z]")
data_TansferMarkt_2021['Position'] = [''.join(index.findall(i)) for i in data_TansferMarkt_2021['Position']]

Position = []
for i in data_TansferMarkt_2021['Position']:
    if i in FB:
        Position.append('FB')
    if i in CB:
        Position.append('CB')
    if i in CM:
        Position.append('CM')
    if i in AM:
        Position.append('AM')
    if i in W:
        Position.append('W')
    if i in CF:
        Position.append('CF')
    if i in GK:
        Position.append('GK')
data_TansferMarkt_2021['Position'] = Position

data_TansferMarkt_2021.to_csv('datasets/new/data_TansferMarkt_2021_new.csv',index = 0)

In [ ]:
data_TansferMarkt_2021[['Player Name', 'Position']].head(5)

### 4.5 Combine Wyscout and Ratings Dataset <a class="anchor" id="sec_4.5"></a>
* [Back to menu](#sec_0)

#### 2021

In [ ]:
SS_Data = pd.read_csv('datasets/data_SofaScore_2021.csv', usecols=['Player','Rating'])
Wy_Data = pd.read_csv('datasets/new/Data_Wy_2021_new.csv')

In [ ]:
SS_Data.head(5)

In [ ]:
Wy_Data.head(5)

In [ ]:
index = re.compile(r"[A-Z]")
SS_Data['Player'][SS_Data['Player']=='Luca David Mayr Falten'] = 'L. Mayr-Fälten'
SS_Data['Player'] = [index.findall(" ".join(i.split(' ')[:-1]))[-1] + '.' + ' ' + i.split(' ')[-1] if len(i.split(' '))>1 else i for i in SS_Data['Player']]

In [ ]:
NewData = pd.merge(Wy_Data, SS_Data, on='Player')

# find out the difference values
Wy_Player = Wy_Data['Player'].to_list()
New_Player = NewData['Player'].to_list()
SS_Player = SS_Data['Player'].to_list()

# In the Wy dataset, but not in the NewData dataset
diffWyNew = [i for i in Wy_Player if i not in New_Player]
# In the SofaScore dataset, but not in the NewData dataset
diffSsNew = [i for i in SS_Player if i not in New_Player]

In [ ]:
diffWyNew

In [ ]:
diffSsNew

In [ ]:
Wy_Data['Player'] = Wy_Data['Player'].replace(['Damia Viader', 'Nil Vinyals', 'J. Carrera', 'Rafael Mentzingen', 'Nicolas Firmino', 'Aime Mabika',
                                               'C. Díaz', 'Sergi Nus', 'C. Ávilez', 'M. Méndez', 'Luis Zamudio', 'Caiser Gomes', 'Shermaine Martina',
                                               'Manuel Ferriol', 'Enric Bernat', 'F. Pérez', 'N.  Greenidge-Duncan', 'E. Vanacore-Decker','R. Gómez',
                                               'T.Johnson', 'Gabriel Morais', 'Y. Öttl', 'Carlos Gomez', 'Ivan Magalhães', 'Pecka', 'B. Taghvai'],
                                              ['D. Viader', 'N. Vinyals', 'J. Carrera-Garcia', 'R. Mentzingen', 'N. Firmino','A. Mabika', 'C. Diaz',
                                               'S. Nus', 'C. Avilez', 'M. Mendez', 'L. Zamudio', 'C. Gomes', 'S. Martina','M. Ferriol', 'E. Bernat',
                                               'F. Perez', 'N. Greenidge-Duncan', 'E. Decker','R. Gomez', 'T. Johnson', 'G. Morais', 'Y. Ottl',
                                               'C. Gomez', 'I. Magalhaes', 'L. Pecka', 'B. Taghvai-Najib'])
SS_Data['Player'] = SS_Data['Player'].replace(['K. ElMedkhar', 'E. Alihodžić', 'G. Kone', 'R. Sommersall', 'R. Hees', 'D. Leon', 'J. España',
                                               'P. Monticelli', 'Y. Galvan', 'C. Gómez', 'T. Kamara', 'B. Toyama'], 
                                              ['K. Elmedkhar', 'E. Alihodzic', 'M. Kone', 'R. Somersall', 'R. van Hees', 'C. De Leon', 'J. Espana', 'J. Monticelli',
                                               'Y. Galvan-Mercado', 'C. Gomez', 'J. Kamara', 'J. Barriga Toyama'])

In [ ]:
NewData = pd.merge(Wy_Data, SS_Data, on='Player')

# find out the difference values
Wy_Player = Wy_Data['Player'].to_list()
New_Player = NewData['Player'].to_list()
SS_Player = SS_Data['Player'].to_list()

# In the Wy dataset, but not in the NewData dataset
diffWyNew = [i for i in Wy_Player if i not in New_Player]
# In the SofaScore dataset, but not in the NewData dataset
diffSsNew = [i for i in SS_Player if i not in New_Player]

In [ ]:
diffWyNew

In [ ]:
diffSsNew

In [ ]:
NewData.to_csv('datasets/data_WyScout_Rating_2021.csv',index = False)

In [ ]:
NewData.head(5)

#### 2020

In [ ]:
Data_SofaScore_2020 = pd.read_csv('datasets/Data_SofaScore_2020.csv', usecols=['Player','Rating'])
Data_Wy_2020 = pd.read_csv('datasets/new/Data_Wy_2020_new.csv')

In [ ]:
Data_SofaScore_2020

In [ ]:
index = re.compile(r"[A-Z]")
Data_SofaScore_2020['Player'][Data_SofaScore_2020['Player']=='Luca David Mayr Falten'] = 'L. Mayr-Fälten'
Data_SofaScore_2020['Player'] = [index.findall(" ".join(i.split(' ')[:-1]))[-1] + '.' + ' ' + i.split(' ')[-1] if len(i.split(' '))>1 else i for i in Data_SofaScore_2020['Player']]

In [ ]:
NewData = pd.merge(Data_Wy_2020, Data_SofaScore_2020, on='Player')

# find out the difference values
Wy_Player = Data_Wy_2020['Player'].to_list()
New_Player = NewData['Player'].to_list()
SS_Player = Data_SofaScore_2020['Player'].to_list()

# In the Wy dataset, but not in the NewData dataset
diffWyNew = [i for i in Wy_Player if i not in New_Player]
# In the SofaScore dataset, but not in the NewData dataset
diffSsNew = [i for i in SS_Player if i not in New_Player]

In [ ]:
diffWyNew

In [ ]:
diffSsNew

In [ ]:
Data_Wy_2020['Player'] = Data_Wy_2020['Player'].replace(['F. Carabalí', 'Damia Viader', 'Paulo Júnior', 'Nil Vinyals', 'Nicolas Firmino', 
                                                         'C. Díaz', 'C. Ávilez', 'M. Méndez', 'Lucas Coutinho', 'R. Zacarías', 'Manuel Ferriol',
                                                         'Luis Zamudio', 'R. Somersall', 'Roberto Alarcón', 'Carlos Gomez', 'A. Gluvačević',
                                                         'Diego Souza', 'Tiago Mendonça', 'R. Gómez', 'M. O’Sullivan', 'Kenji Tanaka',
                                                         'G. Magana Rivera', 'AJ Valenzuela', 'A. Zuluaga', 'E. Ibišević', 'Ivan Magalhães',
                                                         'J. Álvarez', 'P. Botello Faz'],
                                                        ['F. Carabali', 'D. Viader', 'P. Junior', 'N. Vinyals', 'N. Firmino', 'C. Diaz',
                                                         'C. Avilez', 'M. Mendez', 'L. Coutinho', 'R. Zacarias', 'M. Ferriol', 'L. Zamudio',
                                                         'R. Sommersall', 'R. Alarcon', 'C. Gomez', 'A. Gluvacevic','D. Souza',
                                                         'T. Mendonca', 'R. Gomez', "M. O'Sullivan", 'K. Tanaka', 'G. Rivera',
                                                         'A. Valenzuela', 'A. Zuluaga-Silva','E. Ibisevič', 'I. Magalhaes', 'J. Alvarez',
                                                         'P. Faz'])
Data_SofaScore_2020['Player'] = Data_SofaScore_2020['Player'].replace(['J. DeZart', 'C. Gómez', 'C. Banks', 'T. Akinlosotu', 'M. Rivera', 'E. Decker',
                                                                       'P. Monticelli', 'L. Forbes', 'K. Bonilla', 'I. Mare', 'B. Faz', 'R. Godoy',
                                                                       'D. Loach'],
                                                                      ['J. Dezart', 'C. Gomez', 'J. Banks', 'A. Akinlosotu', 'G. Rivera',
                                                                       'E. Vanacore-Decker', 'J. Monticelli', 'J. Forbes','J. Bonilla',
                                                                       'J. Mare', 'P. Faz', 'G. Ramos-Godoy', 'T. DeLoach'])

In [ ]:
NewData = pd.merge(Data_Wy_2020, Data_SofaScore_2020, on='Player')

# find out the difference values
Wy_Player = Data_Wy_2020['Player'].to_list()
New_Player = NewData['Player'].to_list()
SS_Player = Data_SofaScore_2020['Player'].to_list()

# In the Wy dataset, but not in the NewData dataset
diffWyNew = [i for i in Wy_Player if i not in New_Player]
# In the SofaScore dataset, but not in the NewData dataset
diffSsNew = [i for i in SS_Player if i not in New_Player]

In [ ]:
diffWyNew

In [ ]:
diffSsNew

In [ ]:
NewData.to_csv('datasets/data_WyScout_Rating_2020.csv',index = False)

In [ ]:
NewData.head(5)

### 4.6 Data Analysis <a class="anchor" id="sec_4.6"></a>
* [Back to menu](#sec_0)

#### Wyscout Exploratory 

In [ ]:
#Wy_Data_profile = pf.ProfileReport(Wy_Data, title = "Wyscout Data Report 2021")

In [ ]:
#Wy_Data_profile.to_widgets()

In [ ]:
#Wy_Data_profile.to_file('Wyscout Data Report 2021.html')

#### Transfermarkt Exploratory

In [ ]:
# data_Transfer = pd.read_csv('datasets/new/data_TansferMarkt_2021_new.csv')
# Transfermarkt_Data_profile = pf.ProfileReport(data_Transfer, title = "Transfermarkt Data Report 2021")

In [ ]:
# Transfermarkt_Data_profile.to_widgets()

In [ ]:
# Transfermarkt_Data_profile.to_file('Transfermarkt Data Report 2021.html')

#### American Soccer Analysis Exploratory

In [ ]:
# data_ASA = pd.read_csv('datasets/data_ASA.csv')
# ASA_Data_profile = pf.ProfileReport(data_ASA, title = "American Soccer Analysis Data Report")

In [ ]:
# ASA_Data_profile.to_widgets()

In [ ]:
# ASA_Data_profile.to_file('American Soccer Analysis Data Report.html')

### 4.7 Data Visualization <a class="anchor" id="sec_4.7"></a>
* [Back to menu](#sec_0)

In [ ]:
data_Wy = pd.read_csv('datasets/data_WyScout_Rating_2021.csv')
data_Sample = pd.read_csv('datasets/Data_Sample.csv')
data_ASA = pd.read_csv('datasets/data_ASA.csv')

#### Age distribution Wy

In [ ]:
# Age distribution
age = data_Wy['Age'].value_counts()

trace = go.Bar(
    x=age.index,
    y=age.values,
    marker=dict(
        color = age.values,
        colorscale='Reds',
        showscale=True)
)

data = [trace]
layout = go.Layout(
                   paper_bgcolor='rgb(243, 243, 243)',
                   plot_bgcolor='rgb(243, 243, 243)',
                   title='Age distribution', 
                   yaxis = dict(title = 'The number of players')
                  )


fig = go.Figure(data=data, layout=layout)
fig['layout']['xaxis'].update(dict(title = 'Age', 
                                   tickfont = dict(size = 12)))
py.offline.iplot(fig)

In [ ]:
age_avg=data_Wy['Age'].mean()

fig = px.box(data_Wy, y="Team within selected timeframe", x="Age",
            title='<b>Players Age distribution by Team<b>',
            width=750,height=750,template='ggplot2')
fig.add_shape( 
    type="line", line_color="black", line_width=3, opacity=1, line_dash="dot",
    y0=0, y1=1, yref="paper", x0=age_avg, x1=age_avg, xref="x"
)

py.offline.iplot(fig)

#### Player Counting Of Each Teams And Positions ASA

In [ ]:
new_data = pd.DataFrame(data_ASA, columns=['Team','Player','Position'])
data_P = pd.get_dummies(new_data['Position'],prefix='Position')
new_data = pd.concat([new_data, data_P],axis=1)
group = new_data.groupby('Team')[['Position_CB','Position_CM','Position_DM','Position_FB','Position_GK','Position_ST','Position_W']].sum()

In [ ]:
# player Couting Of Each Teams and Positions (Dataset: ASA)

trace0 = go.Bar(
                x = group.index.values,
                y = group.Position_CB,
                name = "Position_CB",
                marker = dict(color = 'rgb(102,255,255)')
                )

trace1 = go.Bar(
                x = group.index.values,
                y = group.Position_CM,
                name = "Position_CM",
                marker = dict(color = 'rgb(102,178,255)')
                )

trace2 = go.Bar(
                x = group.index.values,
                y = group.Position_DM,
                name = "Position_DM",
                marker = dict(color = 'rgb(102,102,255)')
                )

trace3 = go.Bar(
                x = group.index.values,
                y = group.Position_FB,
                name = "Position_FB",
                marker = dict(color = 'rgb(178, 102, 255)')
                )

trace4 = go.Bar(
                x = group.index.values,
                y = group.Position_GK,
                name = "Position_GK",
                marker = dict(color = 'rgb(255, 102, 255)')
                )

trace5 = go.Bar(
                x = group.index.values,
                y = group.Position_ST,
                name = "Position_ST",
                marker = dict(color = 'rgb(255, 255, 178)')
                )

trace6 = go.Bar(
                x = group.index.values,
                y = group.Position_W,
                name = "Position_W",
                marker = dict(color = 'rgb(173, 255, 43)')
                )

data = [trace0,trace1,trace2,trace3,trace4,trace5,trace6]

layout = go.Layout(
                   paper_bgcolor='rgb(243, 243, 243)',
                   plot_bgcolor='rgb(243, 243, 243)',
                   barmode = "group", 
                   title="Player Couting Of Each Teams and Positions",
                   xaxis= dict(title= 'Team',ticklen= 6,zeroline= False), 
                   yaxis= dict(title= 'Count of Players',ticklen= 6,zeroline= False))

fig = go.Figure(data = data, layout = layout)
py.offline.iplot(fig)

#### Sum of Club Player and Sum of Market Value (Wy)

In [ ]:
new_data1 = pd.DataFrame(data_Wy, columns=['Team within selected timeframe'])
new_data1['Count Player'] = data_Wy.groupby('Team within selected timeframe')['Market value'].transform('count')
new_data1['Sum Market Value'] = data_Wy.groupby('Team within selected timeframe')['Market value'].transform('sum')
new_data1 = new_data1.drop_duplicates()

In [ ]:
y_saving = [each for each in new_data1['Count Player']]
y_net_worth  = [float(each) for each in new_data1['Sum Market Value']]
x_saving = [each for each in new_data1['Team within selected timeframe']]
x_net_worth  = [each for each in new_data1['Team within selected timeframe']]

trace0 = go.Bar(
                x=y_saving,
                y=x_saving,
                marker=dict(color='rgba(171, 50, 96, 0.6)',line=dict(color='rgba(171, 50, 96, 1.0)',width=1)),
                name='Count Player',
                orientation='h',
)
trace1 = go.Scatter(
                x=y_net_worth,
                y=x_net_worth,
                mode='lines+markers',
                line=dict(color='rgb(63, 72, 204)'),
                name='Sum Market Value',
)

layout = dict(
                title='Sum of Club Player and Sum of Market Value',
                yaxis=dict(showticklabels=True,domain=[0, 0.85]),
                yaxis2=dict(showline=True,showticklabels=False,linecolor='rgba(102, 102, 102, 0.8)',linewidth=2,domain=[0, 0.85]),
                xaxis=dict(zeroline=False,showline=False,showticklabels=True,showgrid=True,domain=[0, 0.42]),
                xaxis2=dict(zeroline=False,showline=False,showticklabels=True,showgrid=True,domain=[0.47, 1],side='top',dtick=1000000),
                legend=dict(x=0.029,y=1.038,font=dict(size=10) ),
                margin=dict(l=200, r=20,t=70,b=70),
                paper_bgcolor='rgb(243, 243, 243)',
                plot_bgcolor='rgb(243, 243, 243)'
)

fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout'].update(layout)
py.offline.iplot(fig)

#### Relation of All Player Minutes and Player Games ASA

In [ ]:
# Relation of ALL Player Minutes and Player Games (Dataset: ASA)

trace0 =go.Scatter(
                    x = data_ASA[data_ASA.Position == 'W'].Minutes,
                    y = data_ASA[data_ASA.Position == 'W'].Games,
                    mode = "markers",
                    name = "Position of W",
                    marker = dict(color = 'blue'),
                    text= data_ASA[data_ASA.Position == 'W'].Player)

trace1 =go.Scatter(
                    x = data_ASA[data_ASA.Position == 'CM'].Minutes,
                    y = data_ASA[data_ASA.Position == 'CM'].Games,
                    mode = "markers",
                    name = "Position of CM",
                    marker = dict(color = 'red'),
                    text= data_ASA[data_ASA.Position == 'CM'].Player)

trace2 =go.Scatter(
                    x = data_ASA[data_ASA.Position == 'CB'].Minutes,
                    y = data_ASA[data_ASA.Position == 'CB'].Games,
                    mode = "markers",
                    name = "Position of CB",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text= data_ASA[data_ASA.Position == 'CB'].Player)

trace3 =go.Scatter(
                    x = data_ASA[data_ASA.Position == 'DM'].Minutes,
                    y = data_ASA[data_ASA.Position == 'DM'].Games,
                    mode = "markers",
                    name = "Position of DM",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text= data_ASA[data_ASA.Position == 'DM'].Player)

trace4 =go.Scatter(
                    x = data_ASA[data_ASA.Position == 'FB'].Minutes,
                    y = data_ASA[data_ASA.Position == 'FB'].Games,
                    mode = "markers",
                    name = "Position of FB",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text= data_ASA[data_ASA.Position == 'FB'].Player)

trace5 =go.Scatter(
                    x = data_ASA[data_ASA.Position == 'GK'].Minutes,
                    y = data_ASA[data_ASA.Position == 'GK'].Games,
                    mode = "markers",
                    name = "Position of GK",
                    marker = dict(color = 'black'),
                    text= data_ASA[data_ASA.Position == 'GK'].Player)

trace6 =go.Scatter(
                    x = data_ASA[data_ASA.Position == 'ST'].Minutes,
                    y = data_ASA[data_ASA.Position == 'ST'].Games,
                    mode = "markers",
                    name = "Position of ST",
                    marker = dict(color = 'yellow'),
                    text= data_ASA[data_ASA.Position == 'ST'].Player)

data = [trace0,trace1,trace2,trace3,trace4,trace5,trace6]

layout = dict(
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)',
              title = 'Relation of Player Minutes and Player Games',
              xaxis= dict(title= 'Minutes',ticklen= 6,gridcolor='rgb(255, 255, 255)',zeroline= False),
              yaxis= dict(title= 'Games',ticklen= 6,gridcolor='rgb(255, 255, 255)',zeroline= False)
             )

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig) 

#### Country of birth statistics for foreign players (Wy)

In [ ]:
counter_data = data_Wy[['Birth country', 'Matches played']].copy().dropna()
counter_data['Birth country'] = counter_data['Birth country'].replace(['Scotland', 'England', 'St. Kitts and Nevis', 'Tanzania',
                                                                       'St. Vincent and the Grenadines','Republic of Ireland',
                                                                       'Congo DR', 'Venezuela'],
                                                                      ['United Kingdom of Great Britain and Northern Ireland',
                                                                       'United Kingdom of Great Britain and Northern Ireland',
                                                                      'Saint Kitts and Nevis', 'Tanzania, United Republic of',
                                                                       'Saint Vincent and the Grenadines', 'Ireland',
                                                                       'Congo, Democratic Republic of the',
                                                                       'Venezuela (Bolivarian Republic of)'])

#Counter
counter_result = pd.DataFrame(columns = ['name', 'Totall matches played'])
country_list = []
count_cache = []
for i in range(len(counter_data)):
    if counter_data.iloc[i,0] not in country_list:
        country_list.append(counter_data.iloc[i,0])
        count_cache.append(counter_data.iloc[i,1])
    else:
        count_cache[country_list.index(counter_data.iloc[i,0])] += counter_data.iloc[i,1]

counter_result['name'] = country_list
counter_result['Totall matches played'] = count_cache
counter_result = counter_result.sort_values(by = ['Totall matches played']).reset_index(drop=True)

Three_letter_ISO = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

#Merge two dataframe
counter_result_ISO = pd.merge(counter_result, Three_letter_ISO[['name', 'alpha-3']].copy(), on='name')

fig = px.choropleth(counter_result_ISO, locations="alpha-3",
                    color="Totall matches played", # Totall matches played is a column of counter_result_ISO
                    hover_name="Totall matches played", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Teal)
fig.layout.update(title = 'Country of birth statistics for foreign players (Wy)')
fig.show()

## 5. Features Selection (Wyscout Dataset) <a class="anchor" id="sec_5"></a>
* [Back to menu](#sec_0)

In [ ]:
data_Wy = pd.read_csv('datasets/Data_WyScout_Rating_2021.csv')

### 5.1 Split Dataset with Positions <a class="anchor" id="sec_5.1"></a>
* [Back to menu](#sec_0)

After discussing with Oakland Roots, we decide to select different features of each position.

Description of colors (4):  
<font color=red>The most important feature of the location</font>   
<font color=blue>Important features of the location</font>   
<font color=green>Universally important features</font>   
Features with little reference value

- FB: 
    - <font color=red>Successful defensive actions per 90</font>
    - <font color=green> Passes per 90, Accurate passes %, Short/medium passes per 90, Accurate short/medium passes %</font>
    - Goals, Non-penalty goals, Assists, Key passes per 90, Duels per 90, Duels won %, Shots blocked per 90, Interceptions per 90, Crosses per 90, Accurate crosses %, Crosses to goalie box per 90, Received passes per 90, Accurate long passes %, Shot assists per 90, Passes to final third per 90, Accurate passes to final third %

- CB:
    - <font color=red>Successful defensive actions per 90, Duels per 90, Duels won %, Accurate long passes %, Through passes per 90, Accurate through passes %</font>
    - <font color=blue>Passes to final third per 90, Accurate passes to final third %, Dribbles per 90, Successful dribbles </font>
    - <font color=green>Passes per 90, Accurate passes %, Short/medium passes per 90, Accurate short/medium passes %</font>
    - Goals, Non-penalty goals, Assists, Shots blocked per 90, Interceptions per 90, Received passes per 90Key passes per 90
    
- CM:
    - <font color=red>Aerial duels per 90, Touches in box per 90, Average pass length m</font>
    - <font color=blue>Shot assists per 90, Passes to final third per 90, Accurate passes to final third %, Smart Pass per 90, Accurate smart passes %
</font>
    - <font color=green>Passes per 90, Accurate passes %, Short/medium passes per 90, Accurate short/medium passes %</font>
    - Goals, Non-penalty goals, Assists, Duels per 90, Duels won %, Sliding tackles per 90, Interceptions per 90, Received passes per 90, PAdj Interceptions
    
- AM: 
    - <font color=red>Touches in box per 90, Forward passes per 90, Accurate forward passes %, Average pass length m, Second assists per 90</font>
    - <font color=blue>Successful attacking actions per 90, Offensive duels won %, Shot assists per 90, Third assists per 90, Passes to final third per 90, Accurate passes to final third %, Passes to penalty area per 90, Accurate passes to penalty area %</font>
    - <font color=green>Passes per 90, Accurate passes %, Short/medium passes per 90, Accurate short/medium passes %</font>
    - Goals, Non-penalty goals, Assists, Duels per 90, Duels won %, Key passes per 90, Received passes per 90, Key passes per 90
    
- W: 
    - <font color=red>Goals, Non-penalty goals, Assists, Crosses per 90, Accurate crosses %, Crosses to goalie box per 9, Dribbles per 90, Successful dribbles %</font>
    - <font color=blue>Successful attacking actions per 90, Offensive duels won %, Shot assists per 90, Key passes per 90</font>
    - <font color=green>Passes per 90, Accurate passes %, Short / medium passes per 90, Accurate short / medium passes %</font>
    - Key passes per 90, Duels per 90, Duels won %, Shots on target %, Goal conversion %, Received passes per 90, Passes to penalty area per 90, Accurate passes to penalty area %
    
- CF: 
    - <font color=red>Goals, Non-penalty goals, Shots on target %, Goal conversion %, Key passes per 90</font>
    - <font color=blue>Aerial duels per 90, Aerial duels won %, Successful attacking actions per 90, Offensive duels won %,
      Shot assists per 90</font>
    - <font color=green>Passes per 90, Accurate passes %, Short / medium passes per 90, Accurate short / medium passes %</font>
    - Assists, Pass%, Key passes per 90, Duels per 90, Duels won %, Touches in box per 90, Received passes per 90
    
- GK: 
    - <font color=red>Accurate short / medium passes %, Save rate %</font>
    - <font color=blue>Short / medium passes per 90, Shots against</font>
    - Long passes per 90, Accurate long passes %, Conceded goals per 90, Clean sheet

In [ ]:
data_FB = data_Wy[data_Wy['FB'] == 'FB']
data_FB = data_FB[['Goals','Non-penalty goals','Assists','Key passes per 90','Successful defensive actions per 90',
                   'Duels per 90','Duels won, %','Shots blocked per 90','Interceptions per 90','Crosses per 90',
                   'Accurate crosses, %','Crosses to goalie box per 90','Received passes per 90','Passes per 90', 
                   'Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %', 'Accurate long passes, %',
                   'Shot assists per 90','Passes to final third per 90','Accurate passes to final third, %','Rating']]
data_CB = data_Wy[data_Wy['CB'] == 'CB']
data_CB = data_CB[['Goals','Non-penalty goals','Assists','Key passes per 90','Successful defensive actions per 90',
                   'Duels per 90','Duels won, %','Shots blocked per 90','Interceptions per 90','Received passes per 90',
                   'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %',
                   'Accurate long passes, %','Passes to final third per 90','Accurate passes to final third, %',
                   'Dribbles per 90','Successful dribbles, %','Through passes per 90','Accurate through passes, %','Rating']]
data_CM = data_Wy[data_Wy['CM'] == 'CM']
data_CM = data_CM[['Goals','Non-penalty goals','Assists','Duels per 90','Duels won, %','Aerial duels per 90',
                   'Touches in box per 90','Sliding tackles per 90','Interceptions per 90','Received passes per 90',
                   'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %',
                   'Average pass length, m','Shot assists per 90','Passes to final third per 90','Accurate passes to final third, %',
                   'PAdj Interceptions','Smart passes per 90','Accurate smart passes, %','Rating']]
data_AM = data_Wy[data_Wy['AM'] == 'AM']
data_AM = data_AM[['Goals','Non-penalty goals','Assists','Duels per 90','Duels won, %','Key passes per 90',
                   'Touches in box per 90','Successful attacking actions per 90','Offensive duels won, %','Received passes per 90',
                   'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %',
                   'Forward passes per 90','Accurate forward passes, %','Average pass length, m','Shot assists per 90',
                   'Second assists per 90','Third assists per 90','Passes to final third per 90',
                   'Accurate passes to final third, %','Passes to penalty area per 90',
                   'Accurate passes to penalty area, %','Rating']]
data_W = data_Wy[data_Wy['W'] == 'W']
data_W = data_W[['Goals','Non-penalty goals','Assists','Duels per 90','Duels won, %','Shots on target, %',
                 'Successful attacking actions per 90','Goal conversion, %','Crosses per 90',
                 'Accurate crosses, %','Crosses to goalie box per 90','Offensive duels won, %','Received passes per 90',
                 'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %', 
                 'Shot assists per 90','Key passes per 90','Passes to penalty area per 90','Accurate passes to penalty area, %',
                 'Dribbles per 90','Successful dribbles, %','Rating']]
data_CF = data_Wy[data_Wy['CF'] == 'CF']
data_CF = data_CF[['Goals','Non-penalty goals','Assists','Aerial duels per 90',
                   'Aerial duels won, %','Duels per 90','Duels won, %','Successful attacking actions per 90',
                   'Shots on target, %','Goal conversion, %','Offensive duels won, %','Touches in box per 90',
                   'Received passes per 90','Passes per 90','Accurate passes, %','Short / medium passes per 90',
                   'Accurate short / medium passes, %','Shot assists per 90','Key passes per 90','Rating']]
data_GK = data_Wy[data_Wy['GK'] == 'GK']
data_GK = data_GK[['Short / medium passes per 90','Accurate short / medium passes, %','Long passes per 90',
                   'Accurate long passes, %','Conceded goals per 90','Shots against','Clean sheets','Save rate, %', 'Rating']]

### 5.2 Deploy Random Forest Algorithm <a class="anchor" id="sec_5.2"></a>
* [Back to menu](#sec_0)

Using Random Forest Tree to rank the importance of features

In [ ]:
def RandomForest(data):
    
    data_copy = data.copy()
    data_copy["Rating"] = data_copy["Rating"].astype("str")
    data_copy = data_copy.drop(data_copy[data_copy['Rating']=='-'].index)
    Y = data_copy.iloc[:, [-1]]
    Y.reset_index(drop=True,inplace=True)
    X = data_copy.iloc[:, 0:-1]
    X.reset_index(drop=True,inplace=True)

    # sort column index with number of nan value from smallest to largest
    sortindex = np.argsort(X.isnull().sum(axis=0)).values
    
    for i in sortindex:
        # set i column as target 
        feature_i = X.iloc[:, i]
    
        # set other columns as features, including 'y'
        tmp_df = pd.concat([X.iloc[:, X.columns != i], Y], axis=1)
    
        # Fill remaining column missing values with 0
        imp_mf = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
        tmp_df_mf = imp_mf.fit_transform(tmp_df)

        # Use notnull samples in feature_i as training dataset
        y_notnull = feature_i[feature_i.notnull()]
        y_null = feature_i[feature_i.isnull()]   
        X_notnull = tmp_df_mf[y_notnull.index, :]
        X_null = tmp_df_mf[y_null.index, :] 
    
        # continue if this column has no nan value
        if y_null.shape[0] == 0:
            continue

        # RF Regression 
        rfc = RandomForestRegressor(n_estimators=100)
        rfc = rfc.fit(X_notnull, y_notnull)
    
        # predict nan value
        y_predict = rfc.predict(X_null)
    
        # fill
        X.loc[X.iloc[:, i].isnull(), X.columns[i]] = y_predict
        data_copy.loc[data_copy.iloc[:, i].isnull(), data_copy.columns[i]] = y_predict
    
    # split data
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)
    
    global selected_features, feat_lables
    
    feat_lables=data_copy.columns[1:]
    
    forest = RandomForestClassifier(oob_score=True, n_estimators=100, random_state=100, n_jobs=-1)
    forest.fit(x_train, y_train)
    
    # select features which threshold larger then 0.025
    selector = SelectFromModel(forest, threshold=0.025)
    features_important = selector.fit_transform(x_train, y_train)
    model = forest.fit(features_important, y_train)
    selected_features = model.feature_importances_
    
    # print features
    sorted_idx = np.argsort(selected_features)[::-1]
    
    for f in range(len(sorted_idx)):
        print("%2d) %-*s %f"%(f + 1,30,feat_lables[sorted_idx[f]],selected_features[sorted_idx [f]]))    
    

    data0 = pd.DataFrame()
    
    for i in range(len(sorted_idx)):
        data0.insert(i, feat_lables[sorted_idx[i]], data_copy[feat_lables[sorted_idx[i]]], True)
    
    if 'Rating' in data0.columns:
        data0.drop(['Rating'], axis=1, inplace=True)
    else:
        pass
    
    return data0

In [ ]:
position_list = [data_FB, data_CB, data_CM, data_AM, data_W, data_CF, data_GK]
position_name = ['FB', 'CB', 'CM', 'AM', 'W', 'CF', 'GK']
model_name = [0, 1, 2, 3, 4, 5, 6]
num = 0
for i in position_list:
    print('For position', position_name[num])
    model_name[num] = RandomForest(i)
    
    trace = go.Scatter(
        y = selected_features, x = feat_lables.values, mode='markers',
        marker=dict(
            sizemode = 'diameter', sizeref = 1, size = 25, color = selected_features, colorscale='Portland',showscale=True
        ))
    data = [trace]
    layout= go.Layout(
            paper_bgcolor='rgb(243, 243, 243)',
            plot_bgcolor='rgb(243, 243, 243)',
            autosize= True,
            title= 'Random Forest Feature Importance',
            hovermode= 'closest',
            yaxis=dict(title= 'Feature Importance', ticklen= 5,gridwidth= 2),
            showlegend= False
        )
    fig = go.Figure(data = data, layout = layout)
    py.offline.iplot(fig) 
    
    fig = make_subplots(rows=3, cols=1)

    for j in model_name[num].columns[1:]:
        if max(model_name[num][j]) < 5:
            fig.add_trace((go.Box(y=model_name[num][j],name=j)),row=1, col=1)
        elif 6 < max(model_name[num][j]) < 50:
            fig.add_trace((go.Box(y=model_name[num][j],name=j)),row=2, col=1)
        else:
            fig.add_trace((go.Box(y=model_name[num][j],name=j)),row=3, col=1)
            
    updatemenus = list([dict(
    type = "buttons", 
    direction = "left", 
    buttons=list([
            dict(args = ["type", "box"], label = "Box", method = "restyle"),
            dict(args = ["type", "violin"], label = "Violin", method = "restyle")]), 
    pad = {"r": 2, "t": 2},
    showactive = True,
    x = 0.01,
    xanchor = "left",
    y = 1.05,
    yanchor = "top")])

            
    fig.layout.update(
        go.Layout(
        polar = dict(
            radialaxis = dict(
                visible = True,)),
        showlegend = True,
        title = 'Box and Violin Plot of Features',
        updatemenus=updatemenus,
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        height=740
        ))
    
    py.offline.iplot(fig)
    
    num += 1
    
data_FB_Selected = model_name[0]
data_CB_Selected = model_name[1]
data_CM_Selected = model_name[2]
data_AM_Selected = model_name[3]
data_W_Selected = model_name[4]
data_CF_Selected = model_name[5]
data_GK_Selected = model_name[6]

## 6. Cosine Distance <a class="anchor" id="sec_6"></a>
* [Back to menu](#sec_0)

In [ ]:
data_Sample = pd.read_csv('datasets/Data_Sample.csv')
data_Sample

In [ ]:
def Cosine_Distance(Player_data, Position_data):
    
    data_copy = Position_data.copy()
    # select player
    name = data_Sample[data_Sample["Player"] == Player_data]
    # select features in data_sample dataset
    feature = name[[i for i in data_copy.columns.tolist()]]
    
    # normalize
    sclar = preprocessing.StandardScaler()
    data_Norm = pd.DataFrame(sclar.fit_transform(pd.concat([feature, data_copy])))
    
    x = data_Norm.iloc[0:1]
    y = data_Norm.iloc[1:]
    
    cos_s = []
    cos_d = []
    player = []
   
    # cos_sim & cos_dis
    for i in range(len(y)):
        simi = cosine_similarity(x, y[i:i+1]).tolist()[0][0]
        cos_s.append(simi)
        dist = paired_distances(x, y[i:i+1], metric='cosine').tolist()[0]
        cos_d.append(dist)
    
    for i in Position_data.index:
        player.append(data_Wy['Player'][i])
        
    data0 = data_copy
    data0.loc[:, 'Cosine Similarity'] = cos_s
    data0.loc[:, 'Cosine Distances'] = cos_d
    data0.loc[:, 'Player'] = player
    data0 = data0.sort_values(by=['Cosine Similarity'],axis=0,ascending=[False]) 
    
    return data0

In [ ]:
data_JSands = Cosine_Distance('J. Sands', data_CB_Selected)
data_JSands.head(3)

In [ ]:
data_RDamus = Cosine_Distance('R. Damus', data_CF_Selected)
data_RDamus.head(3)

In [ ]:
data_RCannon = Cosine_Distance('R. Cannon', data_FB_Selected)
data_RCannon.head(3)

In [ ]:
data_CBassett = Cosine_Distance('C. Bassett', data_CF_Selected)
data_CBassett.head(3)

### 6.1 Radar Plot <a class="anchor" id="sec_6.1"></a>
* [Back to menu](#sec_0)

In [ ]:
def Polar(Player_data, Selected_Player_data):
    
    data_copy = Selected_Player_data.copy().iloc[0:1,:-3]

    # select player
    name = data_Sample[data_Sample["Player"] == Player_data]
    # select features in data_sample dataset
    feature = name[[i for i in data_copy.columns.tolist()]]
    
    data = pd.concat([feature, data_copy])
    
    fig = make_subplots(rows=1, cols=2,specs=[[{"type": "Polar"},{"type": "Polar"}]])

    R1=[]
    theta1=[]
    R2=[]
    theta2=[]
    R3=[]
    theta3=[]
    R4=[]
    theta4=[]
    
    for i in data.columns[1:]:
        
        if max(data[i]) < 20:
            R1.append(feature[i].iloc[0])
            theta1.append(i)
            R2.append(data_copy[i].iloc[0])
            theta2.append(i)
        else:   
            R3.append(feature[i].iloc[0])
            theta3.append(i)
            R4.append(data_copy[i].iloc[0])
            theta4.append(i)
            
    fig.add_trace(go.Scatterpolar(
          r = R1,
          theta = theta1,
          fill = 'toself',
          marker_color='rgb(47,138,196,30)',
          name = Player_data,
          ),row=1, col=1)
    
    fig.add_trace(go.Scatterpolar(
          r = R2,
          theta = theta2,
          fill = 'toself',
          marker_color='rgb(229,210,245,0.2)',
          name = Selected_Player_data.Player.values[0]),row=1, col=1)
      
    fig.add_trace(go.Scatterpolar(
          r = R3,
          theta = theta3,
          fill = 'toself',
          marker_color='rgb(47,138,196,30)',
          name = Player_data),row=1, col=2)
    
    fig.add_trace(go.Scatterpolar(
          r = R4,
          theta = theta4,
          fill = 'toself',
          marker_color='rgb(229,210,245,0.2)',
          name = Selected_Player_data.Player.values[0]),row=1, col=2)

    fig.layout.update(
        go.Layout(
        polar = dict(
            radialaxis = dict(
                visible = True,)),
        showlegend = True,
        title = "{} vs {}".format(Player_data, Selected_Player_data.Player.values[0]),
        height=400, width=1300,
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)',
        ))
    
    return py.offline.iplot(fig)

In [ ]:
Polar('J. Sands', data_JSands)

In [ ]:
Polar('R. Damus', data_RDamus)

In [ ]:
Polar('R. Cannon', data_RCannon)

In [ ]:
Polar('C. Bassett', data_CBassett)

### 6.1.1 Player comparison engine <a class="anchor" id="sec_6.1.1"></a>
* [Back to menu](#sec_0)

In [ ]:
position_selected_list = [data_FB_Selected, data_CB_Selected, data_CM_Selected, data_AM_Selected, data_W_Selected, data_CF_Selected, data_GK_Selected]
position_name = ['FB', 'CB', 'CM', 'AM', 'W', 'CF', 'GK']
playname_list = [1, 2, 3, 4, 5, 6, 7]


num = 0
for i in position_selected_list:
    player_name = []
    for j in i.index:
        player_name.append(data_Wy['Player'][j])
    playname_list[num] = player_name
    num += 1
    
Name_FB = playname_list[0]
Name_CB = playname_list[1]
Name_CM = playname_list[2]
Name_AM = playname_list[3]
Name_W = playname_list[4]
Name_CF = playname_list[5]
Name_GK = playname_list[6]

In [ ]:
# app = JupyterDash(__name__)

# app.layout = html.Div([
#     dcc.Markdown('''
#         #### Introduction
#         This engine applies to the player ability comparison of `Data_WyScout_Rating_2021.csv`.\n
#         Before using it, you need to select the **type of position** you want to compare first.\n
#         Next, **select the names of the two players separately**. This engine supports search function.\n
#         Finally, click the **submit button** to generate the comparison chart.\n
#         The engine generates interactive images, which can be interacted with by mouse clicks.\n
#     '''),
    
#     html.Div([
#         "Position",
#         dcc.Dropdown(id="position_select",
#                      options = position_name,
#                      placeholder = 'First step: select the position'
#                     ),
#     ]),
#     html.Div([
#         "First player",
#         dcc.Dropdown(id="player1_select",
#                     placeholder = 'Second step: select the first player'
#                     ),
#     ]),
#     html.Div([
#         "Second player",
#         dcc.Dropdown(id="player2_select",
#                     placeholder = 'Third step: select the second player'
#                     ),
#     ]),
#     html.Button('Submit', id='submit-val', n_clicks=0),
#     html.Div(id='container-button-basic',
#              children='Complete the above three steps and press submit'),
#     dcc.Graph(id='radar_plot')
# ])

# @app.callback(
#     [Output("player1_select", "options"),
#      Output("player2_select", "options")],
#     Input("position_select", "value")
# )
# def player_update(value):
#     if not value:
#         raise PreventUpdate
#     elif value == 'FB':
#         return Name_FB, Name_FB
#     elif value == 'CB':
#         return Name_CB, Name_CB
#     elif value == 'CM':
#         return Name_CM, Name_CM
#     elif value == 'AM':
#         return Name_AM, Name_AM
#     elif value == 'W':
#         return Name_W, Name_W
#     elif value == 'CF':
#         return Name_CF, Name_CF
#     elif value == 'GK':
#         return Name_GK, Name_GK
    
# @app.callback(
#     [Output("radar_plot", "figure"),
#     Output("container-button-basic", "children"),
#     Output("submit-val", "n_clicks")],
#     [Input("player1_select", "value"),
#      Input("player2_select", "value"),
#      Input("position_select", "value"),
#     Input("submit-val", "n_clicks")]
# )
# def Graph_update(Player1, Player2, Position, switch):
#     if switch == 0:
#         raise PreventUpdate
#     elif not Player1:
#         raise PreventUpdate
#     elif not Player2:
#         raise PreventUpdate
#     elif not Position:
#         raise PreventUpdate
#     else:
#         pass
    
#     if Position == 'FB':
#         Position_data = data_FB_Selected
#     elif Position == 'CB':
#         Position_data = data_CB_Selected
#     elif Position == 'CM':
#         Position_data = data_CM_Selected
#     elif Position == 'AM':
#         Position_data = data_AM_Selected
#     elif Position == 'W':
#         Position_data = data_W_Selected
#     elif Position == 'CF':
#         Position_data = data_CF_Selected
#     elif Position == 'GK':
#         Position_data = data_GK_Selected
    
#     data_copy = Position_data.copy()
    
#     #Fill back players' name after feature selection
#     player_name = []
#     for i in Position_data.index:
#         player_name.append(data_Wy['Player'][i])
#     data_copy.loc[:, 'Player'] = player_name
    
#     #Positioning input player data and merge them into one dataframe in order to prepare for plot
#     player1_data = data_copy[data_copy['Player'] == Player1].iloc[0:1,: -1]
#     player2_data = data_copy[data_copy['Player'] == Player2].iloc[0:1,: -1]
#     plot_data = pd.concat([player1_data, player2_data])
    
#     #Radar plot
#     fig = make_subplots(rows=1, cols=2,specs=[[{"type": "Polar"},{"type": "Polar"}]])

#     R1=[]
#     theta1=[]
#     R2=[]
#     theta2=[]
#     R3=[]
#     theta3=[]
#     R4=[]
#     theta4=[]
    
#     for i in plot_data.columns:
#         if max(plot_data[i]) < 20:
#             R1.append(plot_data[i].iloc[0])
#             theta1.append(i)
#             R2.append(plot_data[i].iloc[1])
#             theta2.append(i)
#         else:   
#             R3.append(plot_data[i].iloc[0])
#             theta3.append(i)
#             R4.append(plot_data[i].iloc[1])
#             theta4.append(i)
            
#     fig.add_trace(go.Scatterpolar(
#           r = R1,
#           theta = theta1,
#           fill = 'toself',
#           marker_color='rgb(47,138,196,30)',
#           name = Player1),row=1, col=1)
    
#     fig.add_trace(go.Scatterpolar(
#           r = R2,
#           theta = theta2,
#           fill = 'toself',
#           marker_color='rgb(229,210,245,0.2)',
#           name = Player2),row=1, col=1)
    
#     fig.add_trace(go.Scatterpolar(
#           r = R3,
#           theta = theta3,
#           fill = 'toself',
#           marker_color='rgb(47,138,196,30)',
#           name = Player1),row=1, col=2)
    
#     fig.add_trace(go.Scatterpolar(
#           r = R4,
#           theta = theta4,
#           fill = 'toself',
#           marker_color='rgb(229,210,245,0.2)',
#           name = Player2),row=1, col=2)

#     fig.layout.update(
#         go.Layout(
#         polar = dict(
#             radialaxis = dict(
#                 visible = True,)),
#         showlegend = True,
#         title = "{} vs {}".format(Player1, Player2),
#         height=400, width=1200,
#         paper_bgcolor='rgb(243, 243, 243)',
#         plot_bgcolor='rgb(243, 243, 243)',
#         ))
    
#     children = 'Data confirm, graph generated.'
#     switch = 0
#     return fig, children, switch

# if __name__ == '__main__':
#     app.run_server(debug=True)


### 6.2 Another Way Cosine Distance <a class="anchor" id="sec_6.2"></a>
* [Back to menu](#sec_0)

In [ ]:
sample = pd.read_csv('datasets/Data_Sample.csv')

# Position rearrangement
sample['FB'] = ['FB' if len(set(i.replace(' ','').split(',')) & set(FB))>0 else ''  for i in sample['Position']]
sample['CB'] = ['CB' if len(set(i.replace(' ','').split(',')) & set(CB))>0 else ''  for i in sample['Position']]
sample['CM'] = ['CM' if len(set(i.replace(' ','').split(',')) & set(CM))>0 else ''  for i in sample['Position']]
sample['AM'] = ['AM' if len(set(i.replace(' ','').split(',')) & set(AM))>0 else ''  for i in sample['Position']]
sample['W'] = ['W' if len(set(i.replace(' ','').split(',')) & set(W))>0 else ''  for i in sample['Position']]
sample['CF'] = ['CF' if len(set(i.replace(' ','').split(',')) & set(CF))>0 else ''  for i in sample['Position']]
sample['GK'] = ['GK' if len(set(i.replace(' ','').split(',')) & set(GK))>0 else ''  for i in sample['Position']]

# Categorization and merging
sample['Position'] = [",".join([s for s in [sample['FB'][i],sample['CB'][i],sample['CM'][i],sample['AM'][i],\
                                    sample['W'][i],sample['CF'][i],sample['GK'][i]] if s !='']) for i in range(len(sample['Position']))]
# Convert the units of Market value to USD
sample['Market value'] = [i*1.07 for i in sample['Market value']]

sample.to_csv('datasets/new/sample_new.csv',index = 0)

In [ ]:
data_wy_rating_2021 = pd.read_csv('datasets/Data_WyScout_Rating_2021.csv')

data_FB = data_wy_rating_2021[data_wy_rating_2021['Position'] == 'FB']
data_CB = data_wy_rating_2021[data_wy_rating_2021['Position'] == 'CB']
data_CM = data_wy_rating_2021[data_wy_rating_2021['Position'] == 'CM']
data_AM = data_wy_rating_2021[data_wy_rating_2021['Position'] == 'AM']
data_W = data_wy_rating_2021[data_wy_rating_2021['Position'] == 'W']
data_CF = data_wy_rating_2021[data_wy_rating_2021['Position'] == 'CF']
data_GK = data_wy_rating_2021[data_wy_rating_2021['Position'] == 'GK']

In [ ]:
all_features = data_wy_rating_2021.columns.drop(['FB','CB','CM','AM','W','CF','GK','Position','Player','Team','Team within selected timeframe','Market value','Contract expires','Birth country','Passport country','Foot','On loan','Rating'])

# Outlier Fill (mean)
rating_avg = round(data_wy_rating_2021['Rating'][data_wy_rating_2021['Rating']!='-'].astype('float').mean(),1)
data_wy_rating_2021['Rating'].replace('-',6.6,inplace = True)
data_wy_rating_2021['Rating'] = data_wy_rating_2021['Rating'].astype('float')

In [ ]:
features = {}

features['FB'] = ['Assists','Passes per 90','Shots blocked per 90','Shot assists per 90','Successful defensive actions per 90','Crosses per 90','Short / medium passes per 90',
'Crosses to goalie box per 90','Key passes per 90','Duels per 90','Accurate long passes, %','Interceptions per 90','Accurate passes, %','Received passes per 90',
'Passes to final third per 90','Duels won, %','Accurate crosses, %','Accurate short / medium passes, %','Non-penalty goals']

features['CB'] = ['Short / medium passes per 90','Accurate long passes, %','Key passes per 90','Assists','Passes per 90','Successful defensive actions per 90','Interceptions per 90',
'Accurate short / medium passes, %','Shots blocked per 90','Received passes per 90','Dribbles per 90','Passes to final third per 90','Duels won, %',
'Non-penalty goals','Duels per 90','Accurate passes to final third, %','Accurate passes, %','Successful dribbles, %']

features['CM'] = ['Accurate short / medium passes, %','Sliding tackles per 90','Aerial duels per 90','Passes to final third per 90','Accurate passes, %','Average pass length, m',
'Passes per 90','Accurate passes to final third, %','Duels won, %','Duels per 90','PAdj Interceptions','Short / medium passes per 90','Received passes per 90',
'Shot assists per 90','Interceptions per 90','Assists','Touches in box per 90','Smart passes per 90','Non-penalty goals']

features['AM'] = ['Passes to penalty area per 90','Received passes per 90','Passes to final third per 90','Accurate short / medium passes, %','Accurate forward passes, %',
'Forward passes per 90','Short / medium passes per 90','Third assists per 90','Duels won, %','Accurate passes to final third, %','Touches in box per 90',
'Accurate passes, %','Offensive duels won, %','Shot assists per 90','Second assists per 90','Passes per 90','Average pass length, m','Non-penalty goals',
'Successful attacking actions per 90','Key passes per 90','Assists','Duels per 90']

features['W'] = ['Accurate passes, %','Short / medium passes per 90','Offensive duels won, %','Shots on target, %','Accurate passes to penalty area, %','Accurate short / medium passes, %',
'Crosses per 90','Accurate crosses, %','Dribbles per 90','Passes per 90','Received passes per 90','Assists','Successful dribbles, %','Crosses to goalie box per 90',
'Duels won, %','Passes to penalty area per 90','Duels per 90','Goal conversion, %','Non-penalty goals','Shot assists per 90','Successful attacking actions per 90','Key passes per 90']

features['CF'] = ['Touches in box per 90','Shot assists per 90','Goal conversion, %','Successful attacking actions per 90','Short / medium passes per 90','Key passes per 90',
'Shots on target, %','Received passes per 90','Aerial duels won, %','Passes per 90','Duels per 90','Duels won, %','Offensive duels won, %',
'Non-penalty goals','Accurate short / medium passes, %','Assists','Accurate passes, %','Aerial duels per 90']

features['GK'] = ['Conceded goals per 90','Clean sheets','Long passes per 90','Save rate, %','Accurate long passes, %','Accurate short / medium passes, %','Shots against']

#### 6.2.1 Position FB -- Most similar players to R. Cannon <a class="anchor" id="sec_6.2.1"></a>
* [Back to menu](#sec_0)

In [ ]:
# FB
x = data_FB[features['FB']].fillna(-1)
y = sample[features['FB']][sample['Player'] == 'R. Cannon'].fillna(-1)

# Similarity calculation
simi = cosine_similarity(x, y)
simi_data = pd.DataFrame({'Player':data_FB['Player'],'simi':list(simi[:,0])})
FB_Player = list(simi_data.sort_values('simi',ascending=False).head(5)['Player'])
print('Most similar players to R. Cannon - FB:',FB_Player)

#### 6.2.2 Position CB -- Most similar players to J. Sands <a class="anchor" id="sec_6.2.2"></a>
* [Back to menu](#sec_0)

In [ ]:
# CB
x = data_CB[features['CB']].fillna(-1)
y = sample[features['CB']][sample['Player'] == 'J. Sands'].fillna(-1)

# Similarity calculation
simi = cosine_similarity(x, y)
simi_data = pd.DataFrame({'Player':data_CB['Player'],'simi':list(simi[:,0])})
CB_Player = list(simi_data.sort_values('simi',ascending=False).head(5)['Player'])
print('Most similar players to J. Sands - CB:',CB_Player)

#### 6.2.3 Position CM -- Most similar players to J. Sands & C. Bassett <a class="anchor" id="sec_6.2.3"></a>
* [Back to menu](#sec_0)

In [ ]:
# CM
# 2-1
x = data_CM[features['CM']].fillna(-1)
y = sample[features['CM']][sample['Player'] == 'J. Sands'].fillna(-1)

# Similarity calculation
simi = cosine_similarity(x, y)
simi_data = pd.DataFrame({'Player':data_CM['Player'],'simi':list(simi[:,0])})
CM_Player_01 = list(simi_data.sort_values('simi',ascending=False).head(5)['Player'])
print('Most similar players to J. Sands - CM:',CM_Player_01)


# 2-2
x = data_CM[features['CM']].fillna(-1)
y = sample[features['CM']][sample['Player'] == 'C. Bassett'].fillna(-1)

# Similarity calculation
simi = cosine_similarity(x, y)
simi_data = pd.DataFrame({'Player':data_CM['Player'],'simi':list(simi[:,0])})
CM_Player_02 = list(simi_data.sort_values('simi',ascending=False).head(5)['Player'])
print('Most similar players to C. Bassett - CM:',CM_Player_02)

#### 6.2.4 Position AM -- Most similar players to C. Bassett <a class="anchor" id="sec_6.2.4"></a>
* [Back to menu](#sec_0)

In [ ]:
# AM
x = data_AM[features['AM']].fillna(-1)
y = sample[features['AM']][sample['Player'] == 'C. Bassett'].fillna(-1)

# Similarity calculation
simi = cosine_similarity(x, y)
simi_data = pd.DataFrame({'Player':data_AM['Player'],'simi':list(simi[:,0])})
AM_Player = list(simi_data.sort_values('simi',ascending=False).head(5)['Player'])
print('Most similar players to C. Bassett - AM:',AM_Player)

#### 6.2.5 Position CF -- Most similar players to R. Damus & C. Bassett <a class="anchor" id="sec_6.2.5"></a>
* [Back to menu](#sec_0)

In [ ]:
# CF
x = data_CF[features['CF']].fillna(-1)
y = sample[features['CF']][sample['Player'] == 'R. Damus'].fillna(-1)

# 2-1
# Similarity calculation
simi = cosine_similarity(x, y)
simi_data = pd.DataFrame({'Player':data_CF['Player'],'simi':list(simi[:,0])})
CF_Player_01 = list(simi_data.sort_values('simi',ascending=False).head(5)['Player'])
print('Most similar players to R. Damus - CF:',CF_Player_01)

# 2-2
y = sample[features['CF']][sample['Player'] == 'C. Bassett'].fillna(-1)

# Similarity calculation
simi = cosine_similarity(x, y)
simi_data = pd.DataFrame({'Player':data_CF['Player'],'simi':list(simi[:,0])})
CF_Player_02 = list(simi_data.sort_values('simi',ascending=False).head(5)['Player'])
print('Most similar players to C. Bassett - CF:',CF_Player_02)

#### 6.2.6 Summary <a class="anchor" id="sec_6.2.6"></a>
* [Back to menu](#sec_0)

In [ ]:
data_simi_player = pd.DataFrame({'Most similar players to R. Cannon - FB:':FB_Player,'Most similar players to J. Sands - CB:':CB_Player,'Most similar players to J. Sands - CM:':CM_Player_01,'Most similar players to C. Bassett - CM:':CM_Player_02,
 'Most similar players to C. Bassett - AM:':AM_Player,'Most similar players to R. Damus - CF:':CF_Player_01,'Most similar players to C. Bassett - CF:':CF_Player_02})

data_simi_player.to_csv('datasets/new/data_simi_player.csv',index = 0)
data_simi_player

#### 6.2.7 Find Player Information <a class="anchor" id="sec_6.2.7"></a>
* [Back to menu](#sec_0)

In [ ]:
data_TansferMarkt_2021_new = pd.read_csv('datasets/new/data_TansferMarkt_2021_new.csv')
index = re.compile(r"[A-Z]")
data_TansferMarkt_2021_new['Player'] = [index.findall(" ".join(i.split(' ')[:-1]))[-1] + '.' + ' ' + i.split(' ')[-1] if len(i.split(' '))>1 else i for i in data_TansferMarkt_2021_new['Player Name']]

features_Wy = ['Player','Position','Contract expires','Team','Team within selected timeframe','Age','Birth country',
               'Passport country','Foot','Market value']
features_TM = ['Salary', 'Salary Date', 'Highest Market Value', 'Date of Highest Value',
              'Joined Date', 'Contract Until', 'Date of Last Contract Extension', 'Player Agent',
              'Player Name', 'Nat.', 'Age', 'Club', 'Position Detail', 'player_href']

data_simi_player = pd.DataFrame({'Position':['FB']*5+['CB']*5+['CM']*10+['AM']*5+['CF']*10,
                               'Player':FB_Player+CB_Player+CM_Player_01+CM_Player_02+AM_Player+CF_Player_01+CF_Player_02})

tmp = data_Wy[features_Wy]
tmp.columns = ['Player'] + ['Wy_'+i for i in tmp.columns.drop('Player')]

tmp02 = data_TansferMarkt_2021_new[['Player'] + features_TM]
tmp02.columns = ['Player'] + ['TM_'+i for i in features_TM]

data_simi_player = data_simi_player.merge(tmp,on = ['Player'], how = 'left')
data_simi_player = data_simi_player.merge(tmp02,on = ['Player'], how = 'left')
data_simi_player.to_csv('datasets/results/Cosin_Distance_Player.csv')

In [ ]:
data_simi_player.head()

## 7. Rating Model <a class="anchor" id="sec_7"></a>
* [Back to menu](#sec_0)

In [ ]:
Data_Wy_2020 = pd.read_csv('datasets/new/Data_Wy_2020_new.csv')
Data_Wy_2021 = pd.read_csv('datasets/new/Data_Wy_2021_new.csv')
Data_SofaScore_2020 = pd.read_csv('datasets/Data_SofaScore_2020.csv')
Data_SofaScore_2021 = pd.read_csv('datasets/Data_SofaScore_2021.csv')

index = re.compile(r"[A-Z]")
Data_SofaScore_2020['Player'] = [index.findall(" ".join(i.split(' ')[:-1]))[-1] + '.' + ' ' + i.split(' ')[-1] if len(i.split(' '))>1 else i for i in Data_SofaScore_2020['Player']]
Data_SofaScore_2021['Player'] = [index.findall(" ".join(i.split(' ')[:-1]))[-1] + '.' + ' ' + i.split(' ')[-1] if len(i.split(' '))>1 else i for i in Data_SofaScore_2021['Player']]

Data_Wy_2020 = Data_Wy_2020.merge(Data_SofaScore_2020[['Player','Rating']], on = ['Player'], how = 'inner')
Data_Wy_2021 = Data_Wy_2021.merge(Data_SofaScore_2021[['Player','Rating']], on = ['Player'], how = 'inner')

In [ ]:
for p in ['FB','CB','CM','AM','W','CF','GK']:
    # samples from positions
    data_tmp = Data_Wy_2020[Data_Wy_2020['Position'].fillna('').str.contains(p)] 
    
    # training model
    X = data_tmp[features[p]].fillna(-1)
    Y = data_tmp['Rating']
    forest=RandomForestRegressor(
        criterion='mse', max_depth=4, min_samples_split=2, min_samples_leaf=1,
        min_weight_fraction_leaf=0.0, max_features='auto', random_state=0
    )
    forest.fit(X,Y)
    
    # 2021 predictions
    # use positions to choose players
    data_21 = Data_Wy_2021[Data_Wy_2021['Position'].fillna('').str.contains(p)]
    # fill in nan
    data_21['Rating_predict'] = forest.predict(data_21[features[p]].fillna(-1))
    data_21['Rating_diff'] = data_21['Rating_predict'] - data_21['Rating'] #差值，判断标准
    

    # personal info features
    data_21_01 = data_21[features_Wy + ['Rating','Rating_predict','Rating_diff']]
    data_21_01.columns = ['Player'] + ['Wy_'+ i for i in data_21_01.columns.drop('Player')]
    
    data_21_02 = data_TansferMarkt_2021_new[['Player'] + features_TM]
    data_21_02.columns = ['Player'] + ['TM_'+ i for i in features_TM]
    
    data_21 = data_21_01.merge(data_21_02,on = ['Player'], how = 'left')
    
    # save data for prediction
    data_21.to_csv('datasets/new/data_wy_rating_2021_'+ p +'.csv',index = 0)
    
    # prediction value, true value is positive? top5 in descending order
    data_21_top5 = data_21.sort_values('Wy_Rating_diff',ascending = False)
    data_21_top5 = data_21_top5[data_21_top5['Wy_Rating']>0].head(5)
    data_21_top5.to_csv('datasets/results/Top5_rating_2021_' + p +'.csv',index = 0) 

In [ ]:
Rating_result = pd.DataFrame()
for p in ['FB','CB','CM','AM','W','CF','GK']:
    Rating_top_5 = pd.read_csv('datasets/results/Top5_rating_2021_' + p +'.csv')
    Rating_result = pd.concat([Rating_result,Rating_top_5],axis = 0)
Rating_result.to_csv('datasets/results/Rating_Model_Players.csv')

In [ ]:
Rating_result.head()

## 8. Market Value Model <a class="anchor" id="sec_8"></a>
* [Back to menu](#sec_0)

### 8.1 Market Value Forecast <a class="anchor" id="sec_8.1"></a>
* [Back to menu](#sec_0)

In [ ]:
Data_Wy_2020 = pd.read_csv('datasets/new/Data_Wy_2020_new.csv')
Data_Wy_2021 = pd.read_csv('datasets/new/Data_Wy_2021_new.csv')

In [ ]:
for p in ['FB','CB','CM','AM','W','CF','GK']:
    
    # Sample by position of players in 2020
    data_tmp = Data_Wy_2020[Data_Wy_2020['Position'].fillna('').str.contains(p)] 
    
    # train model
    X = data_tmp[all_features].fillna(-1)
    Y = data_tmp['Market value']
    
    forest=RandomForestRegressor(
        criterion='mse', max_depth=2, min_samples_split=2, min_samples_leaf=1,
        min_weight_fraction_leaf=0.0, max_features='auto', random_state=10)
    
    forest.fit(X,Y)
    
    # predict 2021
    data_21 = Data_Wy_2021[Data_Wy_2021['Position'].fillna('').str.contains(p)]
    data_21['Market_value_predict'] = forest.predict(data_21[all_features].fillna(-1))   # fill na
    data_21['Market_value_diff'] = data_21['Market_value_predict'] - data_21['Market value']   # Judgment criteria, predicted value - true value
    
    # save information
    data_21_01 = data_21[features_Wy + ['Market_value_predict','Market_value_diff']]
    data_21_01.columns = ['Player'] + ['Wy_' + i for i in data_21_01.columns.drop('Player')] 
    
    data_21_02 = data_TansferMarkt_2021_new[['Player']+features_TM]
    data_21_02.columns = ['Player'] + ['TM_' + i for i in features_TM]
    
    data_21 = data_21_01.merge(data_21_02,on = ['Player'], how = 'left')
    
    # save prediction
    data_21.to_csv('datasets/results/data_Market_value_2021_' + p +'.csv',index = 0)
    
    # If the predicted value - true value is positive, the top 5 will be ranked in descending order
    data_21_top5 = data_21.sort_values('Wy_Market_value_diff', ascending = False)
    data_21_top5 = data_21_top5[data_21_top5['Wy_Market value']>0].head(5)
    data_21_top5.to_csv('datasets/results/Top5_Market_value_2021_' + p +'.csv',index = 0) 

In [ ]:
MV_result = pd.DataFrame()
for p in ['FB','CB','CM','AM','W','CF','GK']:
    MV_top_5 = pd.read_csv('datasets/results/Top5_Market_value_2021_' + p +'.csv')
    MV_result = pd.concat([MV_result,MV_top_5],axis = 0)
MV_result.to_csv('datasets/results/Market_value_Model_Players.csv')
MV_result.head()

### 8.2 Market Value supplement <a class="anchor" id="sec_8.2"></a>
* [Back to menu](#sec_0)

In [ ]:
data = pd.read_csv('datasets/new/Data_Wy_2021_new.csv')

In [ ]:
Position_dict = {
    'FB': {'Goals','Non-penalty goals','Assists','Key passes per 90','Successful defensive actions per 90',
           'Duels per 90','Duels won, %','Shots blocked per 90','Interceptions per 90','Crosses per 90',
           'Accurate crosses, %','Crosses to goalie box per 90','Received passes per 90','Passes per 90', 
           'Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %', 'Accurate long passes, %',
           'Shot assists per 90','Passes to final third per 90','Accurate passes to final third, %'},
    'CB': {'Goals','Non-penalty goals','Assists','Key passes per 90','Successful defensive actions per 90',
           'Duels per 90','Duels won, %','Shots blocked per 90','Interceptions per 90','Received passes per 90',
           'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %',
           'Accurate long passes, %','Passes to final third per 90','Accurate passes to final third, %',
           'Dribbles per 90','Successful dribbles, %','Through passes per 90','Accurate through passes, %'},
    'CM': {'Goals','Non-penalty goals','Assists','Duels per 90','Duels won, %','Aerial duels per 90',
           'Touches in box per 90','Sliding tackles per 90','Interceptions per 90','Received passes per 90',
           'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %',
           'Average pass length, m','Shot assists per 90','Passes to final third per 90','Accurate passes to final third, %',
           'PAdj Interceptions','Smart passes per 90','Accurate smart passes, %'},
    'AM': {'Goals','Non-penalty goals','Assists','Duels per 90','Duels won, %','Key passes per 90',
           'Touches in box per 90','Successful attacking actions per 90','Offensive duels won, %','Received passes per 90',
           'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %',
           'Forward passes per 90','Accurate forward passes, %','Average pass length, m','Shot assists per 90',
           'Second assists per 90','Third assists per 90','Passes to final third per 90',
           'Accurate passes to final third, %','Passes to penalty area per 90',
           'Accurate passes to penalty area, %'},
    'W': {'Goals','Non-penalty goals','Assists','Duels per 90','Duels won, %','Shots on target, %',
          'Successful attacking actions per 90','Goal conversion, %','Crosses per 90',
          'Accurate crosses, %','Crosses to goalie box per 90','Offensive duels won, %','Received passes per 90',
          'Passes per 90','Accurate passes, %','Short / medium passes per 90','Accurate short / medium passes, %', 
          'Shot assists per 90','Key passes per 90','Passes to penalty area per 90','Accurate passes to penalty area, %',
          'Dribbles per 90','Successful dribbles, %'},
    'CF': {'Goals','Non-penalty goals','Assists','Aerial duels per 90',
           'Aerial duels won, %','Duels per 90','Duels won, %','Successful attacking actions per 90',
           'Shots on target, %','Goal conversion, %','Offensive duels won, %','Touches in box per 90',
           'Received passes per 90','Passes per 90','Accurate passes, %','Short / medium passes per 90',
           'Accurate short / medium passes, %','Shot assists per 90','Key passes per 90'},
    'GK': {'Short / medium passes per 90','Accurate short / medium passes, %','Long passes per 90',
           'Accurate long passes, %','Conceded goals per 90','Shots against','Clean sheets','Save rate, %'}
}

#### 8.2.1 Before supplement <a class="anchor" id="sec_8.2.1"></a>
* [Back to menu](#sec_0)

In [ ]:
data_FB = data[data['FB'] == 'FB']
data_FB_test = data_FB[data_FB['Market value'] == 0]
data_FB_test[['Player', 'Market value']].head(5)

In [ ]:
cache = [0,1,2,3,4,5,6]
num = 0
for position,content in Position_dict.items():
    #data preperarion
    data_position = data[data[position] == position]
    data_position_train = data_position[data_position['Market value'] != 0]
    data_position_test = data_position[data_position['Market value'] == 0]
    
    data_position_train_x = data_position_train[content]
    data_position_train_y = data_position_train[['Market value']]
    
    data_position_test_x = data_position_test[content]
    
    data_position_train_x = data_position_train_x.fillna(value=0)
    data_position_test_x = data_position_test_x.fillna(value=0)
    
    
    #Modeling and forecasting
    rf = RandomForestRegressor(n_estimators=100, max_depth=30, random_state=2)
    knn = KNeighborsRegressor(n_neighbors=20, weights='distance', algorithm='auto', p=2)
    dt = DecisionTreeClassifier(criterion = 'entropy')
    
    rf.fit(data_position_train_x,data_position_train_y)
    knn.fit(data_position_train_x,data_position_train_y)
    dt.fit(data_position_train_x,data_position_train_y)
    
    positon_pr_rf = rf.predict(data_position_test_x)
    positon_pr_knn = knn.predict(data_position_test_x)
    positon_pr_dt = dt.predict(data_position_test_x)
    
    
    #Data operation and backfill
    positon_pr_knn = positon_pr_knn.reshape(len(positon_pr_knn))
    value = np.sum([positon_pr_rf,positon_pr_knn,positon_pr_dt],axis = 0)/3
    data_position_test['Market value'] = value
    
    cache[num] = data_position_test
    num += 1

#### 8.2.2 After supplement <a class="anchor" id="sec_8.2.2"></a>
* [Back to menu](#sec_0)

In [ ]:
cache[0][['Player', 'Market value']].head(5)

### 9. Abandon cases <a class="anchor" id="sec_9"></a>
* [Back to menu](#sec_0)

#### 9.1 The Neural Network Model to Predict the Market Value <a class="anchor" id="sec_9.1"></a>
* [Back to menu](#sec_0)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_csv('datasets/new/Data_Wy_2020_new.csv')

#Screening out Categorical Features
object_cols = [i for i in data.columns if data.dtypes[i]=='object']

data_position = data[data['FB'] == 'FB']
#Trop Categorical Features
data_position = data_position.drop(columns=object_cols)
data_position = data_position.astype('float')

#Drop the data that have more than .3 null
drop_df = data_position.loc[:, data_position.isnull().mean() < .3]

#Fill na with column median
for col in drop_df:
    drop_df.loc[:, col] = drop_df.loc[:, col].fillna(drop_df[col].median())
    
#Normalizing
drop_df.loc[:, drop_df.columns!="Market value"] -= drop_df.loc[:, drop_df.columns!="Market value"].mean()
drop_df.loc[:, drop_df.columns!="Market value"] /= drop_df.loc[:, drop_df.columns!="Market value"].std()
    
#Split the train data and test data
data_train = drop_df[drop_df['Market value'] != 0]
data_test = drop_df[drop_df['Market value'] == 0]

feature_train = data_train.copy().drop(columns=['Market value'])
feature_test = data_train.copy()['Market value']

#Feature Selection - Random Forest
rf = RandomForestClassifier(n_estimators = 100, random_state=123)
rf.fit(feature_train, feature_test)

#Select important features
feature_rankings = pd.DataFrame(rf.feature_importances_, columns = ['importance']).sort_values('importance', ascending=False)
select = SelectFromModel(rf, threshold = 0.01)
select.fit(feature_train, feature_test)
selected_features = []
for important_features in select.get_support(indices=True):
    selected_features.append(feature_train.columns[important_features])
    
selected_features_plus = selected_features
selected_features_plus.append("Market value")
data_train = data_train[selected_features_plus]

#Train/Test Split the train data
X_train = data_train.loc[:data_train.shape[0]*4//5, data_train.columns!="Market value"]
X_test = data_train.loc[:data_train.shape[0]*4//5, ["Market value"]]

#Transpose to get correct dimensions
X_lin_train = X_train_final.to_numpy().T
Y_lin_train = y_train.to_numpy().reshape(-1,1).T

X_lin_test = X_test_final.to_numpy().T
Y_lin_test = y_test.to_numpy().reshape(-1,1).T

In [ ]:
def initialise_parameters(layers_units):
    parameters = {}            # create a dictionary containing the parameters
    for l in range(1, len(layers_units)):
        parameters['W' + str(l)] = 0.001* np.random.randn(layers_units[l],layers_units[l-1])
        parameters['b' + str(l)] = np.zeros((layers_units[l],1))
    return parameters

def sigmoid(z):
    return 1/(1+np.exp(-z))

def relu(z, deriv = False):
    if(deriv):
        return z>0
    else:
        return np.multiply(z, z>0)
    
def forward_propagation(X,parameters,linear):
    cache = {}
    L = len(parameters)//2 #final layer
    cache["A0"] = X #ease of notation since input = layer 0
    for l in range(1, L):
        cache['Z' + str(l)] = np.dot(parameters['W' + str(l)],cache['A' + str(l-1)]) + parameters['b' + str(l)]
        cache['A' + str(l)] = relu(cache['Z' + str(l)])
    #final layer
    cache['Z' + str(L)] = np.dot(parameters['W' + str(L)],cache['A' + str(L-1)]) + parameters['b' + str(L)]
    cache['A' + str(L)] =cache['Z' + str(L)] if linear else sigmoid(cache['Z' + str(L)])
    return cache 

def cost_function(AL,Y, linear):
    m = Y.shape[1]
    if linear:
        cost = (1/(2*m))*(np.sum(np.square(AL-Y)))
    else:
        cost = (-1/m)*( np.sum(np.multiply(Y,np.log(AL))) + np.sum(np.multiply((1-Y),np.log(1-AL))))
    return cost

def backpropagation(cache,Y,parameters):
    L = len(parameters)//2 
    m = Y.shape[1]
    grads = {}
    grads["dZ" + str(L)]= cache["A" + str(L)] - Y
    grads["dW" + str(L)]= (1/m)*np.dot(grads["dZ" + str(L)],cache["A" + str(L-1)].T) 
    grads["db" + str(L)]= (1/m)*np.sum(grads["dZ" + str(L)],axis=1,keepdims=True)
    for l in range(L-1,0,-1):
        grads["dA" + str(l)]= np.dot(parameters["W" + str(l+1)].T,grads["dZ" + str(l+1)])
        grads["dZ" + str(l)]= np.multiply(grads["dA" + str(l)], relu(cache["Z" + str(l)], deriv = True))
        grads["dW" + str(l)]= (1/m)*np.dot(grads["dZ" + str(l)],cache["A" + str(l-1)].T) 
        grads["db" + str(l)]= (1/m)*np.sum(grads["dZ" + str(l)],axis=1,keepdims=True)
    return grads

def F1_score(AL, Y):
    prediction = (AL >= (np.ones_like(AL)/2))
    
    truth_pos = (Y == np.ones_like(Y))
    truth_neg = (Y == np.zeros_like(Y))
    pred_pos = (prediction == np.ones_like(prediction))
    pred_neg = (prediction == np.zeros_like(prediction))

    true_pos = np.sum(np.logical_and(truth_pos,pred_pos))
    if true_pos == 0: #This prevents an undefined computation since precision=recall=0 
        return 0
    false_pos =np.sum(np.logical_and(truth_neg,pred_pos))
    false_neg =np.sum(np.logical_and(truth_pos,pred_neg))
    true_neg =np.sum(np.logical_and(truth_neg,pred_neg))

    precision = true_pos/(true_pos + false_pos)
    recall = (true_pos)/(true_pos + false_neg)
    F1_score = 2*(recall*precision) /(recall + precision)
    return F1_score

def evaluation_metric(AL, Y, linear):
    if linear:
        return cost_function(AL,Y,linear) #MSE for regression
    else:
        return F1_score(AL, Y) #F1 score for classification
    
def train_model(X_train, Y_train,num_epochs,layers_units,learning_rate, linear):
    train_costs = []
    
    parameters = initialise_parameters(layers_units)
    L = len(layers_units)-1 
    for epoch in range (num_epochs):
        #perform one cycle of forward and backward propagation to get the partial derivatives w.r.t. the weights
        #and biases. Calculate the cost - used to monitor training
        cache = forward_propagation(X_train,parameters,linear)
        cost = cost_function(cache["A" + str(L)],Y_train,linear)
        grads = backpropagation(cache,Y_train,parameters)

        #update the parameters using gradient descent
        for l in range(1,L+1):
            parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate*grads["dW" + str(l)]
            parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate*grads["db" + str(l)]

        #periodically output an update on the current cost and performance on the dev set for visualisation
        train_costs.append(cost)
        if(epoch%(num_epochs//10)==0):
            print("Training the model, epoch: " + str(epoch+1))
            print("Cost after epoch " + str((epoch)) + ": " + str(cost))
    print("Training complete!")
    #return the trained parameters and the visualisation metrics
    return parameters, train_costs

def evaluate_model(train_costs,parameters,X_train, Y_train, X_test, Y_test, linear):
    #plot the graphs of training set error
    plt.plot(np.squeeze(train_costs))
    plt.ylabel('Cost')
    plt.xlabel('Iterations')
    plt.title("Training Set Error")
    plt.show()
    L = len(parameters)//2
    
    #For train and test sets, perform a step of forward propagation to obtain the trained model's 
    #predictions and evaluate this with an F1 score.
    train_cache = forward_propagation(X_train,parameters,linear)
    train_AL = train_cache["A"+ str(L)]
    if linear: 
         print("The train set MSE is: "+str(evaluation_metric(train_AL,Y_train, linear)))
    else:
        print("The train set F1 score is: "+str(evaluation_metric(train_AL,Y_train, linear)))
    
    test_cache = forward_propagation(X_test,parameters, linear)
    test_AL = test_cache["A"+ str(L)]
    if linear:
                print("The test set MSE is: "+str(evaluation_metric(test_AL,Y_test, linear)))
    else:
        print("The test set F1 score is: "+str(evaluation_metric(test_AL,Y_test, linear)))

def run_model(X_train, Y_train, X_test, Y_test, hyperparameters, linear=True):
    num_epochs = hyperparameters["num_epochs"]
    layers_units = hyperparameters["layers_units"]
    learning_rate = hyperparameters["learning_rate"]
    
    parameters, train_costs = train_model(X_train, Y_train ,num_epochs,layers_units,learning_rate,linear)         
    evaluate_model(train_costs,parameters,X_train, Y_train, X_test, Y_test,linear)

#define the hyperparameters for the model
def create_hyperparameters(X, num_epochs):
    hyperparameters={}
    hyperparameters["num_epochs"] = num_epochs #number of passes through the training set
    hyperparameters["layers_units"] = [X.shape[0], 64, 64,32, 1] #layer 0 is the input layer
    hyperparameters["learning_rate"] = 1e-3
    return hyperparameters

In [ ]:
hyperparameters = create_hyperparameters(X_lin_train, 1000)
run_model(X_lin_train, Y_lin_train, X_lin_test, Y_lin_test,hyperparameters, linear=True)